In [3]:
import pandas as pd
import numpy as np
import scipy as sp
from tqdm import tqdm
tqdm.pandas()
from dateutil.relativedelta import relativedelta
pd.set_option("display.max_columns", None)
from dateutil.relativedelta import relativedelta
from itertools import accumulate

from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt

# Date format = "%Y-%m-%d" @ datetime type

In [4]:
input_folder_path = r"Individual years data\director_merge_output"
output_folder_path = "director_wrangle_output"
supporting_folder_path = "supporting_datafiles"

## BALANCE CALCULATIONS

### 1 READING DATA AND CLARIFYING DATA TYPES 1

In [5]:
mainDirect0 = pd.read_pickle(rf"{input_folder_path}\Main_Director_incl Cess.pkl")    # Manager level data from 2006 to 2024

In [ ]:
mainDirect0

In [5]:
# Mixed Data Type problematic columns

mixedDataColNum = [15,16,17,29,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,48]
date = [48]
mixedDataColNum2 = [x for x in mixedDataColNum if x not in date]

mixedDataCol = mainDirect0.columns[mixedDataColNum2]
mixedDataCol

Index(['Position on Board', 'Independent (Yes/No)', 'Education1',
       'Cessation Date', 'Other Directorship 2', 'Other Directorship 3',
       'Other Directorship 4', 'Other Directorship 5', 'Other Directorship 6',
       'Other Directorship 7', 'Other Directorship 8', 'Other Directorship 9',
       'Other Directorship 10', 'Other Directorship 11',
       'Other Directorship 12', 'Other Directorship 13',
       'Other Directorship 14', 'Other Directorship 15', 'Brief Profile'],
      dtype='object')

In [6]:
# Mentioning key data types

mainDirect0[mixedDataCol] = mainDirect0[mixedDataCol].astype(str)

mainDirect0["Cessation Reason"] = mainDirect0["Cessation Reason"].astype(str)

mainDirect0["AsOnDate"] = pd.to_datetime( mainDirect0["AsOnDate"], format = "%Y-%m-%d" )
mainDirect0["Date of Birth"] = pd.to_datetime( mainDirect0["Date of Birth"], format = "%d-%b-%Y" )
mainDirect0["Appointment Date"] = pd.to_datetime( mainDirect0["Appointment Date"], format = "%d-%b-%Y" )
mainDirect0["Cessation Date"] = pd.to_datetime( mainDirect0["Cessation Date"], format = "%d-%b-%Y", errors = "coerce" )
mainDirect0["Tenure Valid till"] = pd.to_datetime( mainDirect0["Tenure Valid till"], format = "%d-%b-%Y", errors = "coerce" )

mainDirect0.insert( loc = 3, column = "AsOnYear", value = 0 )
mainDirect0["AsOnYear"] = [x.year for x in mainDirect0["AsOnDate"]]
mainDirect = mainDirect0.copy()

In [7]:
mainDirect

,Symbol,Company,AsOnDate,AsOnYear,ISIN,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Appointment Date,Member of Civil Services,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Occupation,Cessation Date,Cessation Reason,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Brief Profile,Tenure Valid till
0,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027662,MR.,DIPTIMOY,DINESHCHANDRA,BHATTACHARJEE,1955-12-09,M,NaN,2005-04-29,NaN,NaN,nan,nan,GRADUATE: B.SC.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-03-02,NOMINATION ENDED,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"MR.D.D.BHATTACHARJEE, IDBI-NOMINEE DIRECTOR HO...",NaT
1,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027633,MR.,RAMKISHAN,AMIRCHAND,DEVIDAYAL,1950-12-08,M,INDIA,2007-10-13,NONE,NaN,nan,nan,POST GRADUATE: M.B.A.,"POST GRADUATE: M.COM.,MAHARAJA SAYAJIRAO UNIVE...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaN,NaT,nan,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,MR.RAM DEVIDAYAL HAS A MASTER'S DEGREE IN COMM...,NaT
2,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027664,MR.,MIHIR,ARVIND,JOSHI,1977-06-12,M,NaN,2007-08-23,NaN,NaN,nan,nan,"GRADUATE: ASSOCIATE MEMBER,INSTITUTE OF COMPAN...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,nan,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"MR.MIHIR JOSHI, GVFL- NOMINEE DIRECTOR HOLDS D...",NaT
3,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00029006,MR.,ATIL,CHANDRESH,PARIKH,1977-09-04,M,INDIA,2001-01-29,NaN,NaN,nan,nan,"GRADUATE: B.E.-CHEMICAL,GUJARAT UNIVERSITY,AHM...","POST GRADUATE: M.B.A.-FINANCE,CALIFORNIA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CORPORATE GOVERNANCE,F...",NaN,NaT,nan,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"MR. ATIL C. PARIKH, HOLDS A BACHELOR'S DEGREE ...",NaT
4,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027661,LATE MR.,CHANDRESH,SHANTILAL,PARIKH,1946-03-06,M,INDIA,1988-03-29,NaN,NaN,nan,nan,POST GRADUATE: M.SC.-CHEMISTRY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CORPORATE GOVERNANCE,F...",NaN,NaT,nan,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"MR.CHANDRESH S. PARIKH, OUR CHAIRMAN AND MANAG...",NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446904,ZYLOG,ZYLOG SYSTEMS LTD.,2018-03-31,2018,INE225I01026,P00001781,MR.,RAJAGOPAL,NaN,SRINIVASACHARI,1940-03-10,M,INDIA,2007-01-28,NONE,NO,NON-EXECUTIVE DIRECTOR,YES,"GRADUATE: B.COM.,GUJARAT UNIVERSITY,AHMEDABAD",GRADUATE: C.A.I.I.B.(CERTIFIED ASSOCIATE OF IN...,"GRADUATE: DIPLOMA-INDUSTRIAL FINANCE,INDIAN IN...","GRADUATE: LL.B.,GUJARAT UNIVERSITY,AHMEDABAD","POST GRADUATE: M.A.-ECONOMICS,GUJARAT UNIVERSI...",NaN,NaN,NaN,NaN,NaN,"CORPORATE GOVERNANCE,CORPORATE SOCIAL RESPONSI...",RETIRED,2012-12-07,RESIGNED (NO REASON STATED),GMR AIRPORTS LTD.,SREI INFRASTRUCTURE FINANCE LTD.,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"MR.S RAJAGOPAL, FORMER CHAIRMAN & MANAGING DIR...",NaT
446905,ZYLOG,ZYLOG SYSTEMS LTD.,2018-03-31,2018,INE225I01026,P00062851,MR.,K.,S.,SUBRAMANIAN,1943-09-17,M,NaN,2014-02-14,NaN,NO,NON-EXECUTIVE DIRECTOR,YES,POST GRADUATE: F.C.M.A.(FELLOW COST & MANAGEME...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-06-30,RESIGNED (NO REASON STATED),NaN,nan,nan,nan,nan,nan,nan,nan,nan

In [8]:
# mainDirect.loc[ (mainDirect0["Symbol"] == "ASIANELEC") & (mainDirect0["AsOnDate"] == "2006-03-31")]

### 1 INDEP POPULATING 1

In [9]:
# Slicing the dataframe for eefficiency

balDirect = mainDirect[["Symbol", "AsOnDate", "Person Code", "Appointment Date", "Cessation Date", "Independent (Yes/No)"]]
balDirect = balDirect.sort_values(by = ["Symbol", "Person Code", "AsOnDate"]).set_index(["Symbol", "Person Code", "AsOnDate"], drop = False)

In [10]:
# IsIndep populating based on pre existing column as a dummy

balDirect.insert(column = "Indep", loc = 3, value = "U")
balDirect.loc[(balDirect["Independent (Yes/No)"] == "YES"), "Indep"] = "Y"
balDirect.loc[(balDirect["Independent (Yes/No)"] == "NO"), "Indep"] = "N"
balDirect = balDirect.drop("Independent (Yes/No)", axis = 1)

In [11]:
balDirect

Symbol   AsOnDate Person Code Indep  \
Symbol    Person Code AsOnDate                                             
20MICRONS P00004449   2024-03-31  20MICRONS 2024-03-31   P00004449     Y   
          P00027633   2009-03-31  20MICRONS 2009-03-31   P00027633     U   
                      2010-03-31  20MICRONS 2010-03-31   P00027633     U   
                      2011-03-31  20MICRONS 2011-03-31   P00027633     U   
                      2012-03-31  20MICRONS 2012-03-31   P00027633     U   
...                                     ...        ...         ...   ...   
ZYLOG     P00114429   2014-03-31      ZYLOG 2014-03-31   P00114429     U   
                      2015-03-31      ZYLOG 2015-03-31   P00114429     U   
                      2016-03-31      ZYLOG 2016-03-31   P00114429     U   
                      2017-03-31      ZYLOG 2017-03-31   P00114429     U   
                      2018-03-31      ZYLOG 2018-03-31   P00114429     U   

                                 Appointment Date Cessation Date  
Symbol    Person Code AsOnDate                                    
20MICRONS P00004449   2024-03-31       2023-05-16            NaT  
          P00027633   2009-03-31       2007-10-13            NaT  
                      2010-03-31       2007-10-13            NaT  
                      2011-03-31       2007-10-13            NaT  
                      2012-03-31       2007-10-13            NaT  
...                                           ...            ...  
ZYLOG     P00114429   2014-03-31       2003-12-30            NaT  
                      2015-03-31       2003-12-30            NaT  
                      2016-03-31       2003-12-30            NaT  
                      2017-03-31       2003-12-30            NaT  
                      2018-03-31       2003-12-30            NaT  

[446909 rows x 6 columns]

### 1 PREVIOUS LAST SERVED, NEXT SERVED AND HANDLING CESSATION and APPOINTMENT DATES 5

In [12]:
# prevLastServed = previous record
# nextServed = Next record

balDirect = balDirect.reset_index(drop = True).sort_values(["Symbol", "Person Code", "AsOnDate"])
prevLastServed = balDirect.groupby(["Symbol", "Person Code"])["AsOnDate"].shift( periods = 1 )
nextServed = balDirect.groupby(["Symbol", "Person Code"])["AsOnDate"].shift( periods = -1 )

balDirect["PrevLastServed"] = prevLastServed
balDirect["NextServed"] = nextServed

In [13]:
balDirect

,Symbol,AsOnDate,Person Code,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed
0,20MICRONS,2024-03-31,P00004449,Y,2023-05-16,NaT,NaT,NaT
1,20MICRONS,2009-03-31,P00027633,U,2007-10-13,NaT,NaT,2010-03-31
2,20MICRONS,2010-03-31,P00027633,U,2007-10-13,NaT,2009-03-31,2011-03-31
3,20MICRONS,2011-03-31,P00027633,U,2007-10-13,NaT,2010-03-31,2012-03-31
4,20MICRONS,2012-03-31,P00027633,U,2007-10-13,NaT,2011-03-31,2013-03-31
...,...,...,...,...,...,...,...,...
446904,ZYLOG,2014-03-31,P00114429,U,2003-12-30,NaT,2013-03-31,2015-03-31
446905,ZYLOG,2015-03-31,P00114429,U,2003-12-30,NaT,2014-03-31,2016-03-31
446906,ZYLOG,2016-03-31,P00114429,U,2003-12-30,NaT,2015-03-31,2017-03-31
446907,ZYLOG,2017-03-31,P00114429,U,2003-12-30,NaT,2016-03-31,2018-03-31


#### Cessation Dates and Dummy

In [14]:
# Cessation dates only start occuring, on the year of resignation and after, therefore, we try to identify those places 
# where there is a prev last served (if not, we shouldn't remove it as its the only record) and where there is a cessation date for removal
# However, Cessation dates stop appearing from the year a director is reappointed. But, when that happens, the Appointment Date
# doesnt' change. This data is ASS

# Adding cessation dates as AsOnDate - 6 months where there is no next served and no cessation date

mask = (pd.isnull(balDirect["NextServed"]))&(pd.isnull(balDirect["Cessation Date"]))
balDirect.loc[ mask, "Cessation Date"] = [x for x in balDirect.loc[mask]["AsOnDate"]]
balDirect = balDirect.sort_values(["Symbol", "Person Code", "AsOnDate"])

balDirect["CessationDummy"] = np.where( pd.notnull(balDirect["Cessation Date"]) , 1, 0)
CessationCumDummy = balDirect.groupby( ["Symbol", "Person Code", "Cessation Date"]).CessationDummy.cumsum()
balDirect["CessationCumDummy"] = CessationCumDummy.fillna(0)
balDirect2 = balDirect.loc[ ~ (balDirect["CessationCumDummy"] > 1)].copy()

#### Appointment Dates and Dummy

In [15]:
ReappointDummy = balDirect2.groupby( ["Symbol", "Person Code"]).CessationDummy.shift(periods = 1)

balDirect2.loc[:, "ReappointDummy"] = ReappointDummy
balDirect2.loc[ balDirect2["ReappointDummy"] == 1, "Appointment Date"] = [x - relativedelta(months = 6) for x in 
                                                                          balDirect2.loc[ balDirect2["ReappointDummy"] == 1, "AsOnDate"]]
# na only occurs at the first value, where they have been appointed
balDirect2["TermStartDummy"] = balDirect2["ReappointDummy"].fillna(1)
balDirect2["ReappointDummy"] = balDirect2["ReappointDummy"].fillna(0)

In [16]:
def ReappointmentDate(frame):
    frame = frame.reset_index(drop = True)
    for i in range(len(frame)):
        if frame.iloc[i] < frame.iloc[i-1]:
            frame.iloc[i] = frame.iloc[i-1]

    return frame        

In [17]:
AppointmentDate = balDirect2.sort_values(["Symbol", "Person Code", "AsOnDate"]).groupby(
    ["Symbol", "Person Code"])["Appointment Date"].progress_apply( ReappointmentDate )

100%|███████████████████████████████████████████████████████████████████████████| 50632/50632 [03:05<00:00, 273.49it/s]


In [18]:
AppointmentDate.reset_index(drop = False).drop(["level_2"], axis = 1).set_index(["Symbol", "Person Code"])

Appointment Date
Symbol    Person Code                 
20MICRONS P00004449         2023-05-16
          P00027633         2007-10-13
          P00027633         2007-10-13
          P00027633         2007-10-13
          P00027633         2007-10-13
...                                ...
ZYLOG     P00114429         2012-09-30
          P00114429         2012-09-30
          P00114429         2012-09-30
          P00114429         2012-09-30
          P00114429         2012-09-30

[259418 rows x 1 columns]

In [19]:
balDirect3 = balDirect2.copy()
balDirect3.set_index(["Symbol", "Person Code"]).loc[:,"Appointment Date"] = AppointmentDate.reset_index(drop = False).drop(["level_2"], axis = 1).set_index(["Symbol", "Person Code"])
balDirect3.loc[:, "TermNumber"] = balDirect2.groupby(["Symbol", "Person Code"])["TermStartDummy"].cumsum()
balDirect3["AppointDummy"] = balDirect3.progress_apply(lambda row: 1 if (row["AsOnDate"] - row["Appointment Date"]).days < 366 else 0, axis = 1)
balDirect3 = balDirect3.dropna(subset = "Appointment Date")
balDirect3

100%|███████████████████████████████████████████████████████████████████████| 259418/259418 [00:03<00:00, 73520.06it/s]


,Symbol,AsOnDate,Person Code,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,CessationCumDummy,ReappointDummy,TermStartDummy,TermNumber,AppointDummy
0,20MICRONS,2024-03-31,P00004449,Y,2023-05-16,2024-03-31,NaT,NaT,1,1.0,0.0,1.0,1.0,1
1,20MICRONS,2009-03-31,P00027633,U,2007-10-13,NaT,NaT,2010-03-31,0,0.0,0.0,1.0,1.0,0
2,20MICRONS,2010-03-31,P00027633,U,2007-10-13,NaT,2009-03-31,2011-03-31,0,0.0,0.0,0.0,1.0,0
3,20MICRONS,2011-03-31,P00027633,U,2007-10-13,NaT,2010-03-31,2012-03-31,0,0.0,0.0,0.0,1.0,0
4,20MICRONS,2012-03-31,P00027633,U,2007-10-13,NaT,2011-03-31,2013-03-31,0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446904,ZYLOG,2014-03-31,P00114429,U,2003-12-30,NaT,2013-03-31,2015-03-31,0,0.0,0.0,0.0,2.0,0
446905,ZYLOG,2015-03-31,P00114429,U,2003-12-30,NaT,2014-03-31,2016-03-31,0,0.0,0.0,0.0,2.0,0
446906,ZYLOG,2016-03-31,P00114429,U,2003-12-30,NaT,2015-03-31,2017-03-31,0,0.0,0.0,0.0,2.0,0
446907,ZYLOG,2017-03-31,P00114429,U,2003-12-30,NaT,2016-03-31,2018-03-31,0,0.0,0.0,0.0,2.0,0


In [20]:
balDirect3.loc[ (balDirect3["Symbol"] == "CINEMAXIN") & (balDirect3["AsOnDate"] == "2013-03-31")]
# cases where cessation date being 6 months in the term causes problem

,Symbol,AsOnDate,Person Code,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,CessationCumDummy,ReappointDummy,TermStartDummy,TermNumber,AppointDummy
79033,CINEMAXIN,2013-03-31,P00000928,N,2013-01-08,2013-01-30,NaT,NaT,1,1.0,0.0,1.0,1.0,1
79034,CINEMAXIN,2013-03-31,P00002478,Y,2013-03-28,2013-03-31,NaT,NaT,1,1.0,0.0,1.0,1.0,1
79035,CINEMAXIN,2013-03-31,P00002481,Y,2013-01-17,2013-03-31,NaT,NaT,1,1.0,0.0,1.0,1.0,1
79036,CINEMAXIN,2013-03-31,P00007182,N,2013-01-08,2013-03-31,NaT,NaT,1,1.0,0.0,1.0,1.0,1
79037,CINEMAXIN,2013-03-31,P00021069,U,2013-01-30,2013-03-31,NaT,NaT,1,1.0,0.0,1.0,1.0,1
79038,CINEMAXIN,2013-03-31,P00027281,Y,2013-01-08,2013-03-31,NaT,NaT,1,1.0,0.0,1.0,1.0,1
79039,CINEMAXIN,2013-03-31,P00042797,Y,2013-03-21,2013-03-31,NaT,NaT,1,1.0,0.0,1.0,1.0,1
79040,CINEMAXIN,2013-03-31,P00045245,U,2013-01-30,2013-03-31,NaT,NaT,1,1.0,0.0,1.0,1.0,1


In [21]:
balDirect3.loc[ (balDirect3["Symbol"] == "ASIANELEC") & (balDirect3["AsOnDate"] == "2006-03-31")]

,Symbol,AsOnDate,Person Code,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,CessationCumDummy,ReappointDummy,TermStartDummy,TermNumber,AppointDummy
30319,ASIANELEC,2006-03-31,P00000310,U,1999-09-23,NaT,NaT,2007-03-31,0,0.0,0.0,1.0,1.0,0
30334,ASIANELEC,2006-03-31,P00003700,U,2003-11-25,NaT,NaT,2007-03-31,0,0.0,0.0,1.0,1.0,0


In [22]:
# balDirect2.loc[ balDirect2["Person Code"] == "P00033271" ]


In [23]:
# balDirect2.loc[ balDirect2["Person Code"] == "P00114429" ]

In [24]:
# balDirect2.loc[ balDirect2["Person Code"] == "P00075399" ]

In [25]:
balDirect5 = balDirect3.drop(["CessationCumDummy"], axis = 1).copy()

### 5 BALANCE VAR INITIALIZATION AND CUMULATIVE SUM USAGE 6

In [26]:
# Instantiating and assigning Opening and Closing Balance variables
# Closing Balance is the balance on AsOnDate, Opening Balance is the balance on one year before AsOnDate, or the balance with which the current year opens.
# Consider this like a balance sheet being filed on AsOnDate, your opening balancde will be how your year started.
# Opening bal variables bring data from previous rows.

balDirect5["CumOpBalUnc"] = 0.0
balDirect5["CumOpBalIndep"] = 0.0
balDirect5["CumOpBalNonIndep"] = 0.0

balDirect5["CumCloBalUnc"] = 0.0
balDirect5["CumCloBalIndep"] = 0.0
balDirect5["CumCloBalNonIndep"] = 0.0

balDirect5["TermOpBalUnc"] = 0.0
balDirect5["TermOpBalIndep"] = 0.0
balDirect5["TermOpBalNonIndep"] = 0.0
balDirect5["TermOpBalTotal"] = 0.0

balDirect5["TermCloBalUnc"] = 0.0
balDirect5["TermCloBalIndep"] = 0.0
balDirect5["TermCloBalNonIndep"] = 0.0
balDirect5["TermCloBalTotal"] = 0.0

balDirect5["CompOpBalUnc"] = 0.0
balDirect5["CompOpBalIndep"] = 0.0
balDirect5["CompOpBalNonIndep"] = 0.0
balDirect5["CompOpBalTotal"] = 0.0

balDirect5["CompCloBalUnc"] = 0.0
balDirect5["CompCloBalIndep"] = 0.0
balDirect5["CompCloBalNonIndep"] = 0.0
balDirect5["CompCloBalTotal"] = 0.0
balDirect5

,Symbol,AsOnDate,Person Code,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,AppointDummy,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal
0,20MICRONS,2024-03-31,P00004449,Y,2023-05-16,2024-03-31,NaT,NaT,1,0.0,1.0,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20MICRONS,2009-03-31,P00027633,U,2007-10-13,NaT,NaT,2010-03-31,0,0.0,1.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20MICRONS,2010-03-31,P00027633,U,2007-10-13,NaT,2009-03-31,2011-03-31,0,0.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20MICRONS,2011-03-31,P00027633,U,2007-10-13,NaT,2010-03-31,2012-03-31,0,0.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20MICRONS,2012-03-31,P00027633,U,2007-10-13,NaT,2011-03-31,2013-03-31,0,0.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446904,ZYLOG,2014-03-31,P00114429,U,2003-12-30,NaT,2013-03-31,2015-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
446905,ZYLOG,2015-03-31,P00114429,U,2003-12-30,NaT,2014-03-31,2016-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
446906,ZYLOG,2016-03-31,P00114429,U,2003-12-30,NaT,2015-03-31,2017-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
446907,ZYLOG,2017-03-31,P00114429,U,2003-12-30,NaT,2016-03-31,2018-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
mask1 = pd.isnull(balDirect5["PrevLastServed"])
mask2 = balDirect5["TermStartDummy"] == 1
mask3 = balDirect5["CessationDummy"] == 1

# initializing the unlogged unclear year balance data
balDirect5.loc[ mask1 & mask3, "CumCloBalUnc"] = [ x.days for x in
                                                  balDirect5.loc[ mask1 & mask3, "Cessation Date"] - balDirect5.loc[ mask1 & mask3, "Appointment Date"]
                                                  ]
balDirect5.loc[ mask1 & (~mask3), "CumCloBalUnc"] = [ x.days for x in
                                                     balDirect5.loc[ mask1 & (~mask3), "AsOnDate"] - balDirect5.loc[ mask1 & (~mask3), "Appointment Date"]
                                                     ]

# Cessation Date - Appointment Date
balDirect5.loc[ ((~mask1) & (mask3) & (mask2)) & (balDirect5["Indep"] == "U"), "CumCloBalUnc"] += [x.days for x in
            balDirect5.loc[ ((~mask1) & (mask3) & (mask2)) & (balDirect5["Indep"] == "U"), "Cessation Date"] - balDirect5.loc[ ((~mask1) & (mask3) & (mask2)) & (balDirect5["Indep"] == "U"), "Appointment Date"]
                                                                                          ]

balDirect5.loc[ ((~mask1) & (mask3) & (mask2)) & (balDirect5["Indep"] == "Y"), "CumCloBalIndep"] += [x.days for x in
            balDirect5.loc[ ((~mask1) & (mask3) & (mask2)) & (balDirect5["Indep"] == "Y"), "Cessation Date"] - balDirect5.loc[ ((~mask1) & (mask3) & (mask2)) & (balDirect5["Indep"] == "Y"), "Appointment Date"]
                                                                                            ]

balDirect5.loc[ ((~mask1) & (mask3) & (mask2)) & (balDirect5["Indep"] == "N"), "CumCloBalNonIndep"] += [x.days for x in
            balDirect5.loc[ ((~mask1) & (mask3) & (mask2)) & (balDirect5["Indep"] == "N"), "Cessation Date"] - balDirect5.loc[ ((~mask1) & (mask3) & (mask2)) & (balDirect5["Indep"] == "N"), "Appointment Date"]
                                                                                               ]

# Cessation Date - PrevLastServed
balDirect5.loc[ ((~mask1) & (mask3) & (~mask2)) & (balDirect5["Indep"] == "U"), "CumCloBalUnc"] += [x.days for x in
            balDirect5.loc[ ((~mask1) & (mask3) & (~mask2)) & (balDirect5["Indep"] == "U"), "Cessation Date"] - balDirect5.loc[ ((~mask1) & (mask3) & (~mask2)) & (balDirect5["Indep"] == "U"), "PrevLastServed"]
                                                                                          ]

balDirect5.loc[ ((~mask1) & (mask3) & (~mask2)) & (balDirect5["Indep"] == "Y"), "CumCloBalIndep"] += [x.days for x in
            balDirect5.loc[ ((~mask1) & (mask3) & (~mask2)) & (balDirect5["Indep"] == "Y"), "Cessation Date"] - balDirect5.loc[ ((~mask1) & (mask3) & (~mask2)) & (balDirect5["Indep"] == "Y"), "PrevLastServed"]
                                                                                            ]

balDirect5.loc[ ((~mask1) & (mask3) & (~mask2)) & (balDirect5["Indep"] == "N"), "CumCloBalNonIndep"] += [x.days for x in
            balDirect5.loc[ ((~mask1) & (mask3) & (~mask2)) & (balDirect5["Indep"] == "N"), "Cessation Date"] - balDirect5.loc[ ((~mask1) & (mask3) & (~mask2)) & (balDirect5["Indep"] == "N"), "PrevLastServed"]
                                                                                               ]

# AsOnDate - Appointment Date
balDirect5.loc[ ((~mask1) & (~mask3) & (mask2)) & (balDirect5["Indep"] == "U"), "CumCloBalUnc"] += [x.days for x in
            balDirect5.loc[ ((~mask1) & (~mask3) & (mask2)) & (balDirect5["Indep"] == "U"), "AsOnDate"] - balDirect5.loc[ ((~mask1) & (~mask3) & (mask2)) & (balDirect5["Indep"] == "U"), "Appointment Date"]
                                                                                          ]

balDirect5.loc[ ((~mask1) & (~mask3) & (mask2)) & (balDirect5["Indep"] == "Y"), "CumCloBalIndep"] += [x.days for x in
            balDirect5.loc[ ((~mask1) & (~mask3) & (mask2)) & (balDirect5["Indep"] == "Y"), "AsOnDate"] - balDirect5.loc[ ((~mask1) & (~mask3) & (mask2)) & (balDirect5["Indep"] == "Y"), "Appointment Date"]
                                                                                            ]

balDirect5.loc[ ((~mask1) & (~mask3) & (mask2)) & (balDirect5["Indep"] == "N"), "CumCloBalNonIndep"] += [x.days for x in
            balDirect5.loc[ ((~mask1) & (~mask3) & (mask2)) & (balDirect5["Indep"] == "N"), "AsOnDate"] - balDirect5.loc[ ((~mask1) & (~mask3) & (mask2)) & (balDirect5["Indep"] == "N"), "Appointment Date"]
                                                                                               ]

# AsOnDate - PrevLastServed
balDirect5.loc[ ((~mask1) & (~mask3) & (~mask2)) & (balDirect5["Indep"] == "U"), "CumCloBalUnc"] += [x.days for x in
            balDirect5.loc[ ((~mask1) & (~mask3) & (~mask2)) & (balDirect5["Indep"] == "U"), "AsOnDate"] - balDirect5.loc[ ((~mask1) & (~mask3) & (~mask2)) & (balDirect5["Indep"] == "U"), "PrevLastServed"]
                                                                                          ]

balDirect5.loc[ ((~mask1) & (~mask3) & (~mask2)) & (balDirect5["Indep"] == "Y"), "CumCloBalIndep"] += [x.days for x in
            balDirect5.loc[ ((~mask1) & (~mask3) & (~mask2)) & (balDirect5["Indep"] == "Y"), "AsOnDate"] - balDirect5.loc[ ((~mask1) & (~mask3) & (~mask2)) & (balDirect5["Indep"] == "Y"), "PrevLastServed"]
                                                                                            ]

balDirect5.loc[ ((~mask1) & (~mask3) & (~mask2)) & (balDirect5["Indep"] == "N"), "CumCloBalNonIndep"] += [x.days for x in
            balDirect5.loc[ ((~mask1) & (~mask3) & (~mask2)) & (balDirect5["Indep"] == "N"), "AsOnDate"] - balDirect5.loc[ ((~mask1) & (~mask3) & (~mask2)) & (balDirect5["Indep"] == "N"), "PrevLastServed"]
                                                                                               ]


balDirect5["TermCloBalUnc"] = balDirect5.groupby(["Symbol", "Person Code", "TermNumber"])["CumCloBalUnc"].cumsum()
balDirect5["TermCloBalIndep"] = balDirect5.groupby(["Symbol", "Person Code", "TermNumber"])["CumCloBalIndep"].cumsum()
balDirect5["TermCloBalNonIndep"] = balDirect5.groupby(["Symbol", "Person Code", "TermNumber"])["CumCloBalNonIndep"].cumsum()

balDirect5["TermOpBalUnc"] = balDirect5["TermCloBalUnc"]
balDirect5["TermOpBalIndep"] = balDirect5["TermCloBalIndep"]
balDirect5["TermOpBalNonIndep"] = balDirect5["TermCloBalNonIndep"]

balDirect5.loc[ balDirect5["Indep"] == "U","TermOpBalUnc"] -= (365)
balDirect5.loc[ balDirect5["Indep"] == "Y","TermOpBalIndep"] -= (365)
balDirect5.loc[ balDirect5["Indep"] == "N","TermOpBalNonIndep"] -= (365)

# Handling those cases where opening balance would be zero
balDirect5.loc[ (balDirect5["TermOpBalUnc"] < 0),"TermOpBalUnc"] = 0
balDirect5.loc[ (balDirect5["TermOpBalIndep"] < 0),"TermOpBalIndep"] = 0
balDirect5.loc[ (balDirect5["TermOpBalNonIndep"] < 0),"TermOpBalNonIndep"] = 0

balDirect5["TermOpBalTotal"] = balDirect5["TermOpBalUnc"] + balDirect5["TermOpBalIndep"] + balDirect5["TermOpBalNonIndep"]
balDirect5["TermCloBalTotal"] = balDirect5["TermCloBalUnc"] + balDirect5["TermCloBalIndep"] + balDirect5["TermCloBalNonIndep"]

In [28]:
# company level balance calculations

balDirect5["CompCloBalUnc"] = balDirect5.groupby(["Symbol", "Person Code"])["CumCloBalUnc"].cumsum()
balDirect5["CompCloBalIndep"] = balDirect5.groupby(["Symbol", "Person Code"])["CumCloBalIndep"].cumsum()
balDirect5["CompCloBalNonIndep"] = balDirect5.groupby(["Symbol", "Person Code"])["CumCloBalNonIndep"].cumsum()

balDirect5["CompOpBalUnc"] = balDirect5["CompCloBalUnc"]
balDirect5["CompOpBalIndep"] = balDirect5["CompCloBalIndep"]
balDirect5["CompOpBalNonIndep"] = balDirect5["CompCloBalNonIndep"]

balDirect5.loc[ (balDirect5["Indep"] == "U"),"CompOpBalUnc"] -= (365)
balDirect5.loc[ (balDirect5["Indep"] == "Y"),"CompOpBalIndep"] -= (365)
balDirect5.loc[ (balDirect5["Indep"] == "N"),"CompOpBalNonIndep"] -= (365)

# Handling those cases where opening balance would be zero
balDirect5.loc[ (balDirect5["CompOpBalUnc"] < 0),"CompOpBalUnc"] = 0
balDirect5.loc[ (balDirect5["CompOpBalIndep"] < 0),"CompOpBalIndep"] = 0
balDirect5.loc[ (balDirect5["CompOpBalNonIndep"] < 0),"CompOpBalNonIndep"] = 0

balDirect5["CompOpBalTotal"] = balDirect5["CompOpBalUnc"] + balDirect5["CompOpBalIndep"] + balDirect5["CompOpBalNonIndep"]
balDirect5["CompCloBalTotal"] = balDirect5["CompCloBalUnc"] + balDirect5["CompCloBalIndep"] + balDirect5["CompCloBalNonIndep"]

balDirect5

,Symbol,AsOnDate,Person Code,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,AppointDummy,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal
0,20MICRONS,2024-03-31,P00004449,Y,2023-05-16,2024-03-31,NaT,NaT,1,0.0,1.0,1.0,1,0.0,0.0,0.0,320.0,0.0,0.0,320.0,0.0,0.0,320.0,320.0,0.0,0.0,320.0,320.0,0.0,0.0,320.0,320.0,0.0,0.0,320.0
1,20MICRONS,2009-03-31,P00027633,U,2007-10-13,NaT,NaT,2010-03-31,0,0.0,1.0,1.0,0,0.0,0.0,0.0,535.0,0.0,0.0,170.0,0.0,0.0,170.0,535.0,0.0,0.0,535.0,170.0,0.0,0.0,170.0,535.0,0.0,0.0,535.0
2,20MICRONS,2010-03-31,P00027633,U,2007-10-13,NaT,2009-03-31,2011-03-31,0,0.0,0.0,1.0,0,0.0,0.0,0.0,365.0,0.0,0.0,535.0,0.0,0.0,535.0,900.0,0.0,0.0,900.0,535.0,0.0,0.0,535.0,900.0,0.0,0.0,900.0
3,20MICRONS,2011-03-31,P00027633,U,2007-10-13,NaT,2010-03-31,2012-03-31,0,0.0,0.0,1.0,0,0.0,0.0,0.0,365.0,0.0,0.0,900.0,0.0,0.0,900.0,1265.0,0.0,0.0,1265.0,900.0,0.0,0.0,900.0,1265.0,0.0,0.0,1265.0
4,20MICRONS,2012-03-31,P00027633,U,2007-10-13,NaT,2011-03-31,2013-03-31,0,0.0,0.0,1.0,0,0.0,0.0,0.0,366.0,0.0,0.0,1266.0,0.0,0.0,1266.0,1631.0,0.0,0.0,1631.0,1266.0,0.0,0.0,1266.0,1631.0,0.0,0.0,1631.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446904,ZYLOG,2014-03-31,P00114429,U,2003-12-30,NaT,2013-03-31,2015-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,365.0,0.0,0.0,182.0,0.0,0.0,182.0,547.0,0.0,0.0,547.0,1704.0,0.0,0.0,1704.0,2069.0,0.0,0.0,2069.0
446905,ZYLOG,2015-03-31,P00114429,U,2003-12-30,NaT,2014-03-31,2016-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,365.0,0.0,0.0,547.0,0.0,0.0,547.0,912.0,0.0,0.0,912.0,2069.0,0.0,0.0,2069.0,2434.0,0.0,0.0,2434.0
446906,ZYLOG,2016-03-31,P00114429,U,2003-12-30,NaT,2015-03-31,2017-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,366.0,0.0,0.0,913.0,0.0,0.0,913.0,1278.0,0.0,0.0,1278.0,2435.0,0.0,0.0,2435.0,2800.0,0.0,0.0,2800.0
446907,ZYLOG,2017-03-31,P00114429,U,2003-12-30,NaT,2016-03-31,2018-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,365.0,0.0,0.0,1278.0,0.0,0.0,1278.0,1643.0,0.0,0.0,1643.0,2800.0,0.0,0.0,2800.0,3165.0,0.0,0.0,3165.0


In [29]:
# balDirect5.loc[ balDirect5["Person Code"] == "P00033271" ]

In [30]:
# balDirect5.loc[ balDirect5["Person Code"] == "P00114429" ]

In [31]:
# balDirect5.loc[ balDirect5["Person Code"] == "P00075399" ]

In [32]:
# balDirect5.loc[ (balDirect5["Person Code"] == "P00004449") & (balDirect5["Symbol"] == "APCOTEXIND") ].sort_values("AsOnDate")

### [NOT IN USE]

In [33]:
# # cumulative sum to capture cumulative years of Independent status within a company for a person
# # initiate cumsum values with 1 wrt their correspondant independent status, these will cumulative summed over later

# balDirect5.loc[ balDirect5["Indep"] == "U","cumsum_U"] = 1
# balDirect5.loc[ balDirect5["Indep"] == "Y","cumsum_Y"] = 1
# balDirect5.loc[ balDirect5["Indep"] == "N","cumsum_N"] = 1

# # The following handles those fractional years where AsOnDate and Appintment/Cessation dates are less than a year apart

# balDirect5.loc[ mask , "cumsum_U"] *= daysDiffAppoint.loc[mask]
# balDirect5.loc[ mask , "cumsum_Y"] *= daysDiffAppoint.loc[mask]
# balDirect5.loc[ mask , "cumsum_N"] *= daysDiffAppoint.loc[mask]

# daysDiffCess = (365 - (balDirect5["AsOnDate"] - balDirect5["Cessation Date"]).dt.days) / 365
# mask = ((daysDiffCess < 1) & (daysDiffCess >= 0))

# balDirect5.loc[ mask , "cumsum_U"] *= daysDiffCess.loc[mask]
# balDirect5.loc[ mask , "cumsum_Y"] *= daysDiffCess.loc[mask]
# balDirect5.loc[ mask , "cumsum_N"] *= daysDiffCess.loc[mask]

# balDirect5

In [34]:
# # Rearranging the Datafile

# balDirect5cols = balDirect5.columns
# balDirect5cols_rearranged = ['Symbol',  'Person Code', 'AsOnDate', 'PrevLastServed', 'Indep', 'Appointment Date', 'Cessation Date',  
#                              'PrevDummy', 'PrevCumDummy', 'ReappointDummy', 'cumsum_U', 'cumsum_Y', 'cumsum_N',
#                              'OpBalUnc', 'OpBalIndep', 'OpBalNonIndep', 'OpBalTotal',
#                              'CloBalUnc', 'CloBalIndep', 'CloBalNonIndep','CloBalTotal',
#                             ]
# balDirect6 = balDirect5[balDirect5cols_rearranged].reset_index(drop = True)
# balDirect6

In [35]:
# # Assigns Opening Balance Unclear with number of days of term before any record. Since there are no records,
# # these days are under unclear independent status

# # Here, this variable is not showcasing the status as of closing current year, but is showing opening current year, therefore,
# # AsOnDate - 1 year
# # The current years" term days will be added later and opening balance will be inferred afterwards.

# # cumulative sum function works on cumsum columns

# def OpBalUncInitial(frame):

#     frame["OpBalUnc"] += ((frame.loc[:, "AsOnDate"].min() - pd.Timedelta(days = 365))- frame["Appointment Date"].min()).days 

#     frame["cumsum_U"] = np.cumsum(frame["cumsum_U"])
#     frame["cumsum_Y"] = np.cumsum(frame["cumsum_Y"])
#     frame["cumsum_N"] = np.cumsum(frame["cumsum_N"])
#     return (frame)

In [36]:
# balDirect6 = balDirect5.reset_index(drop = True).groupby(
#     ["Symbol", "Person Code"] ).progress_apply( OpBalUncInitial )

In [37]:
# balDirect6

In [38]:
# # Opening Blance Claculations

# balDirect6["OpBalUnc"] = balDirect6["CloBalUnc"]
# balDirect6["OpBalIndep"] = balDirect6["CloBalIndep"]
# balDirect6["OpBalNonIndep"] = balDirect6["CloBalNonIndep"]

# balDirect6.loc[ balDirect6["Indep"] == "U","OpBalUnc"] -= (365)
# balDirect6.loc[ balDirect6["Indep"] == "Y","OpBalIndep"] -= (365)
# balDirect6.loc[ balDirect6["Indep"] == "N","OpBalNonIndep"] -= (365)

# # removing negative values
# balDirect6["OpBalUnc"] = np.where(balDirect6["OpBalUnc"]>0, balDirect6["OpBalUnc"], 0)
# balDirect6["OpBalIndep"] = np.where(balDirect6["OpBalIndep"]>0, balDirect6["OpBalIndep"], 0)
# balDirect6["OpBalNonIndep"] = np.where(balDirect6["OpBalNonIndep"]>0, balDirect6["OpBalNonIndep"], 0)


# balDirect6["OpBalTotal"] = balDirect6["OpBalUnc"] + balDirect6["OpBalIndep"] + balDirect6["OpBalNonIndep"]
# balDirect6["CloBalTotal"] = balDirect6["CloBalUnc"] + balDirect6["CloBalIndep"] + balDirect6["CloBalNonIndep"]

In [39]:
# # Adds 365 days based on current rows cumulative terms as dictated by cumsum columns
# # cumsum carries forward the terms throughout the symbolxPCode combination

# balDirect6["OpBalUnc"] = np.where( balDirect6["OpBalUnc"] > 0, balDirect6["OpBalUnc"], 0)

# balDirect6["CloBalUnc"] = balDirect6["OpBalUnc"]

# balDirect6["CloBalUnc" ] += (balDirect6["cumsum_U"]) * 365   
# balDirect6["CloBalIndep" ] += (balDirect6["cumsum_Y"]) * 365  
# balDirect6["CloBalNonIndep" ] += (balDirect6["cumsum_N"]) * 365

### 7 Total Experience 8

In [40]:
totalXPFrame = pd.DataFrame( columns = ["Person Code"] + [f"{year}-03-31" for year in range(1940, 2025)])

In [41]:
totalXPFrame

,Person Code,1940-03-31,1941-03-31,1942-03-31,1943-03-31,1944-03-31,1945-03-31,1946-03-31,1947-03-31,1948-03-31,1949-03-31,1950-03-31,1951-03-31,1952-03-31,1953-03-31,1954-03-31,1955-03-31,1956-03-31,1957-03-31,1958-03-31,1959-03-31,1960-03-31,1961-03-31,1962-03-31,1963-03-31,1964-03-31,1965-03-31,1966-03-31,1967-03-31,1968-03-31,1969-03-31,1970-03-31,1971-03-31,1972-03-31,1973-03-31,1974-03-31,1975-03-31,1976-03-31,1977-03-31,1978-03-31,1979-03-31,1980-03-31,1981-03-31,1982-03-31,1983-03-31,1984-03-31,1985-03-31,1986-03-31,1987-03-31,1988-03-31,1989-03-31,1990-03-31,1991-03-31,1992-03-31,1993-03-31,1994-03-31,1995-03-31,1996-03-31,1997-03-31,1998-03-31,1999-03-31,2000-03-31,2001-03-31,2002-03-31,2003-03-31,2004-03-31,2005-03-31,2006-03-31,2007-03-31,2008-03-31,2009-03-31,2010-03-31,2011-03-31,2012-03-31,2013-03-31,2014-03-31,2015-03-31,2016-03-31,2017-03-31,2018-03-31,2019-03-31,2020-03-31,2021-03-31,2022-03-31,2023-03-31,2024-03-31


In [42]:
# EMPTY SEQUENCES ARE FALSE

def TotalXP(frame):
    
    frameSort = frame.sort_values(by = ["Appointment Date", "Cessation Date"], ascending = [True, False])            
    frameFilt = frameSort.drop_duplicates( subset = ["Appointment Date"], keep = "first")
    appCess = set(zip(frameFilt["Appointment Date"], frameFilt["Cessation Date"]))
    
    relDict = {}
    for year in range(1940, 2025):
        asOnDate = pd.to_datetime( f"{year}-03-31", format = "%Y-%m-%d")
        relDict[f"rel{year}"] = [
            (-(x[0] - asOnDate).days) if pd.notnull(x[1]) and asOnDate < x[1]
            else (365-(asOnDate - x[1]).days) if pd.notnull(x[1]) and (asOnDate - x[1]).days < 365
            else 0 if pd.notnull(x[1]) and (asOnDate - x[1]).days > 365
            else (-(x[0] - asOnDate).days)
            for x in appCess if x[0] < asOnDate]
          
        # relDict[f"rel{year}"] = [(-(x[0] - (x[1] if (pd.notnull(x[1]) and (x[1] - asOnDate).days < 365) else x[0])
        #                             if (pd.notnull(x[1]) and (x[1] < asOnDate) else asOnDate))).days
        #                          if (x[0] < asOnDate) else 0
        #                          for x in appCess]


    output = frame["Person Code"].unique().tolist() + [ (max(relDict[relyear], default = 0) )
                                                       if max(relDict[relyear], default = 0) < 365 else 365
                                                       for relyear in relDict ]
    return output

In [43]:
totalXPList = balDirect5.groupby("Person Code")[["Person Code", "AsOnDate", "Appointment Date", "Cessation Date"]]\
    .progress_apply(TotalXP)

100%|████████████████████████████████████████████████████████████████████████████| 35290/35290 [14:59<00:00, 39.23it/s]


In [44]:
totalXPFrame = pd.DataFrame( columns = ["Person Code"] + [f"{year}-03-31" for year in range(1940, 2025)], data = totalXPList.to_list())
totalXPFrame

,Person Code,1940-03-31,1941-03-31,1942-03-31,1943-03-31,1944-03-31,1945-03-31,1946-03-31,1947-03-31,1948-03-31,1949-03-31,1950-03-31,1951-03-31,1952-03-31,1953-03-31,1954-03-31,1955-03-31,1956-03-31,1957-03-31,1958-03-31,1959-03-31,1960-03-31,1961-03-31,1962-03-31,1963-03-31,1964-03-31,1965-03-31,1966-03-31,1967-03-31,1968-03-31,1969-03-31,1970-03-31,1971-03-31,1972-03-31,1973-03-31,1974-03-31,1975-03-31,1976-03-31,1977-03-31,1978-03-31,1979-03-31,1980-03-31,1981-03-31,1982-03-31,1983-03-31,1984-03-31,1985-03-31,1986-03-31,1987-03-31,1988-03-31,1989-03-31,1990-03-31,1991-03-31,1992-03-31,1993-03-31,1994-03-31,1995-03-31,1996-03-31,1997-03-31,1998-03-31,1999-03-31,2000-03-31,2001-03-31,2002-03-31,2003-03-31,2004-03-31,2005-03-31,2006-03-31,2007-03-31,2008-03-31,2009-03-31,2010-03-31,2011-03-31,2012-03-31,2013-03-31,2014-03-31,2015-03-31,2016-03-31,2017-03-31,2018-03-31,2019-03-31,2020-03-31,2021-03-31,2022-03-31,2023-03-31,2024-03-31
0,P00000005,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,211,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365
1,P00000006,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,59,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,164,0,0
2,P00000007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,219,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,34,0,0,0,0,0,0
3,P00000008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,59,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365
4,P00000009,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,354,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,365,179,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35285,P00123356,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,365
35286,P00123357,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,179,365
35287,P00123448,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,238,365,365
35288,P00123948,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,189,365,365


In [45]:
totalXPFrame2 = totalXPFrame.melt(id_vars = "Person Code", var_name = "AsOnDate", 
                                  value_name = "CloTotalXP").sort_values(["Person Code", "AsOnDate"]).reset_index(drop = True).drop_duplicates()
totalXPFrame2["AsOnDate"] = pd.to_datetime(totalXPFrame2["AsOnDate"], format = "%Y-%m-%d")

In [46]:
CloTotalXPcum = totalXPFrame2.groupby(by = "Person Code").CloTotalXP.progress_apply( np.cumsum )

100%|██████████████████████████████████████████████████████████████████████████| 35290/35290 [00:10<00:00, 3297.70it/s]


In [47]:
CloTotalXPcum.reset_index().drop(["level_1", "Person Code"], axis = 1).rename({"CloTotalXP":"CloBalTotalXP"}, axis = 1)

,CloBalTotalXP
0,0
1,0
2,0
3,0
4,0
...,...
2999645,0
2999646,0
2999647,215
2999648,580


In [48]:
totalXPFrame3 = totalXPFrame2.merge(CloTotalXPcum.reset_index().drop(["level_1", "Person Code"], axis = 1)\
                                    .rename({"CloTotalXP":"CloBalTotalXP"}, axis = 1),
                                    left_index = True, right_index = True)
totalXPFrame3

,Person Code,AsOnDate,CloTotalXP,CloBalTotalXP
0,P00000005,1940-03-31,0,0
1,P00000005,1941-03-31,0,0
2,P00000005,1942-03-31,0,0
3,P00000005,1943-03-31,0,0
4,P00000005,1944-03-31,0,0
...,...,...,...,...
2999645,P00123949,2020-03-31,0,0
2999646,P00123949,2021-03-31,0,0
2999647,P00123949,2022-03-31,215,215
2999648,P00123949,2023-03-31,365,580


In [49]:
balDirect6 = balDirect5.merge(totalXPFrame3, on = ["Person Code", "AsOnDate"], how = "left")\
    .drop("CloTotalXP", axis = 1)
balDirect6

,Symbol,AsOnDate,Person Code,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,AppointDummy,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal,CloBalTotalXP
0,20MICRONS,2024-03-31,P00004449,Y,2023-05-16,2024-03-31,NaT,NaT,1,0.0,1.0,1.0,1,0.0,0.0,0.0,320.0,0.0,0.0,320.0,0.0,0.0,320.0,320.0,0.0,0.0,320.0,320.0,0.0,0.0,320.0,320.0,0.0,0.0,320.0,10224
1,20MICRONS,2009-03-31,P00027633,U,2007-10-13,NaT,NaT,2010-03-31,0,0.0,1.0,1.0,0,0.0,0.0,0.0,535.0,0.0,0.0,170.0,0.0,0.0,170.0,535.0,0.0,0.0,535.0,170.0,0.0,0.0,170.0,535.0,0.0,0.0,535.0,2799
2,20MICRONS,2010-03-31,P00027633,U,2007-10-13,NaT,2009-03-31,2011-03-31,0,0.0,0.0,1.0,0,0.0,0.0,0.0,365.0,0.0,0.0,535.0,0.0,0.0,535.0,900.0,0.0,0.0,900.0,535.0,0.0,0.0,535.0,900.0,0.0,0.0,900.0,3164
3,20MICRONS,2011-03-31,P00027633,U,2007-10-13,NaT,2010-03-31,2012-03-31,0,0.0,0.0,1.0,0,0.0,0.0,0.0,365.0,0.0,0.0,900.0,0.0,0.0,900.0,1265.0,0.0,0.0,1265.0,900.0,0.0,0.0,900.0,1265.0,0.0,0.0,1265.0,3529
4,20MICRONS,2012-03-31,P00027633,U,2007-10-13,NaT,2011-03-31,2013-03-31,0,0.0,0.0,1.0,0,0.0,0.0,0.0,366.0,0.0,0.0,1266.0,0.0,0.0,1266.0,1631.0,0.0,0.0,1631.0,1266.0,0.0,0.0,1266.0,1631.0,0.0,0.0,1631.0,3894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258252,ZYLOG,2014-03-31,P00114429,U,2003-12-30,NaT,2013-03-31,2015-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,365.0,0.0,0.0,182.0,0.0,0.0,182.0,547.0,0.0,0.0,547.0,1704.0,0.0,0.0,1704.0,2069.0,0.0,0.0,2069.0,3742
258253,ZYLOG,2015-03-31,P00114429,U,2003-12-30,NaT,2014-03-31,2016-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,365.0,0.0,0.0,547.0,0.0,0.0,547.0,912.0,0.0,0.0,912.0,2069.0,0.0,0.0,2069.0,2434.0,0.0,0.0,2434.0,4107
258254,ZYLOG,2016-03-31,P00114429,U,2003-12-30,NaT,2015-03-31,2017-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,366.0,0.0,0.0,913.0,0.0,0.0,913.0,1278.0,0.0,0.0,1278.0,2435.0,0.0,0.0,2435.0,2800.0,0.0,0.0,2800.0,4472
258255,ZYLOG,2017-03-31,P00114429,U,2003-12-30,NaT,2016-03-31,2018-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,365.0,0.0,0.0,1278.0,0.0,0.0,1278.0,1643.0,0.0,0.0,1643.0,2800.0,0.0,0.0,2800.0,3165.0,0.0,0.0,3165.0,4837


In [50]:
# balDirect6.loc[ (balDirect6["Person Code"] == "P00004449") & (balDirect6["Symbol"] == "APCOTEXIND") ].sort_values("AsOnDate")

In [51]:
# balDirect6.loc[ balDirect6["Person Code"] == "P00033271" ].sort_values("AsOnDate")

In [52]:
# balDirect6.loc[ balDirect6["Person Code"] == "P00000006" ].sort_values("AsOnDate")

In [53]:
# balDirect6.loc[ balDirect6["Person Code"] == "P00114429" ].sort_values("AsOnDate")

In [54]:
# Errors in data
balDirect6.loc[\
    (balDirect6[["CumOpBalUnc", "CumOpBalIndep", "CumOpBalNonIndep", "CumCloBalUnc", "CumCloBalIndep", "CumCloBalNonIndep",
                "TermOpBalUnc", "TermOpBalIndep", "TermOpBalNonIndep", "TermOpBalTotal", "TermCloBalUnc", "TermCloBalIndep",
                "TermCloBalNonIndep", "TermCloBalTotal", "CompOpBalUnc", "CompOpBalIndep", "CompOpBalNonIndep", "CompOpBalTotal",
                "CompCloBalUnc", "CompCloBalIndep", "CompCloBalNonIndep", "CompCloBalTotal", "CloBalTotalXP"]]<0).any(axis = 1)]

,Symbol,AsOnDate,Person Code,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,AppointDummy,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal,CloBalTotalXP
6367,AGRITECH,2024-03-31,P00044086,N,2023-09-30,2023-07-15,2023-03-31,NaT,1,1.0,1.0,2.0,1,0.0,0.0,0.0,0.0,0.0,-77.0,0.0,0.0,0.0,0.0,0.0,0.0,-77.0,-77.0,1668.0,0.0,1368.0,3036.0,1668.0,0.0,1733.0,3401.0,6031
15684,ARE&M,2013-03-31,P00024717,N,2007-06-22,2008-08-14,2012-03-31,2014-03-31,1,0.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,-1325.0,1744.0,0.0,0.0,1744.0,1744.0,0.0,-1325.0,419.0,1744.0,0.0,0.0,1744.0,1744.0,0.0,-1325.0,419.0,2108
15685,ARE&M,2020-03-31,P00024717,N,2019-09-30,NaT,2019-03-31,2021-03-31,0,1.0,1.0,2.0,1,0.0,0.0,0.0,0.0,0.0,183.0,0.0,0.0,0.0,0.0,0.0,0.0,183.0,183.0,1744.0,0.0,0.0,1744.0,1744.0,0.0,-1142.0,602.0,4663
15686,ARE&M,2021-03-31,P00024717,N,2007-06-22,NaT,2020-03-31,2022-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,0.0,0.0,365.0,0.0,0.0,183.0,183.0,0.0,0.0,548.0,548.0,1744.0,0.0,0.0,1744.0,1744.0,0.0,-777.0,967.0,5028
15687,ARE&M,2022-03-31,P00024717,N,2007-06-22,2021-06-12,2021-03-31,2023-03-31,1,0.0,0.0,2.0,0,0.0,0.0,0.0,0.0,0.0,73.0,0.0,0.0,256.0,256.0,0.0,0.0,621.0,621.0,1744.0,0.0,0.0,1744.0,1744.0,0.0,-704.0,1040.0,5393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244721,VAIBHAVGBL,2023-03-31,P00022584,N,1989-05-08,NaT,2022-03-31,2024-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,0.0,0.0,365.0,0.0,0.0,547.0,547.0,0.0,0.0,912.0,912.0,8363.0,0.0,0.0,8363.0,8363.0,0.0,-93.0,8270.0,12372
246197,VEDL,2013-03-31,P00028097,N,2007-10-30,2009-03-30,2012-03-31,2014-03-31,1,0.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,-1097.0,1614.0,0.0,0.0,1614.0,1614.0,0.0,-1097.0,517.0,1614.0,0.0,0.0,1614.0,1614.0,0.0,-1097.0,517.0,1978
246198,VEDL,2022-03-31,P00028097,Y,2021-09-30,NaT,2021-03-31,2023-03-31,0,1.0,1.0,2.0,1,0.0,0.0,0.0,0.0,182.0,0.0,0.0,0.0,0.0,0.0,0.0,182.0,0.0,182.0,1614.0,0.0,0.0,1614.0,1614.0,182.0,-1097.0,699.0,5263
246199,VEDL,2023-03-31,P00028097,Y,2007-10-30,NaT,2022-03-31,2024-03-31,0,0.0,0.0,2.0,0,0.0,0.0,0.0,0.0,365.0,0.0,0.0,182.0,0.0,182.0,0.0,547.0,0.0,547.0,1614.0,182.0,0.0,1796.0,1614.0,547.0,-1097.0,1064.0,5628


### 1 MERGE WITH MAIN 2

In [55]:
mainDirect2 = mainDirect.merge(balDirect6.reset_index(drop = True), on = ["Symbol", "Person Code", "AsOnDate"], how = "inner", validate = "1:1")
mainDirect2 = mainDirect2.drop(["Appointment Date_x", "Cessation Date_x"], axis = 1).rename({"Appointment Date_y":"Appointment Date", "Cessation Date_y":"Cessation Date"}, axis = 1)
mainDirect2

,Symbol,Company,AsOnDate,AsOnYear,ISIN,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Member of Civil Services,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Occupation,Cessation Reason,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Brief Profile,Tenure Valid till,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,AppointDummy,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal,CloBalTotalXP
0,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027662,MR.,DIPTIMOY,DINESHCHANDRA,BHATTACHARJEE,1955-12-09,M,NaN,NaN,NaN,nan,nan,GRADUATE: B.SC.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOMINATION ENDED,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"MR.D.D.BHATTACHARJEE, IDBI-NOMINEE DIRECTOR HO...",NaT,U,2005-04-29,2009-03-02,NaT,2010-03-31,1,0.0,1.0,1.0,0,0.0,0.0,0.0,1403.0,0.0,0.0,1038.0,0.0,0.0,1038.0,1403.0,0.0,0.0,1403.0,1038.0,0.0,0.0,1038.0,1403.0,0.0,0.0,1403.0,1402
1,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027633,MR.,RAMKISHAN,AMIRCHAND,DEVIDAYAL,1950-12-08,M,INDIA,NONE,NaN,nan,nan,POST GRADUATE: M.B.A.,"POST GRADUATE: M.COM.,MAHARAJA SAYAJIRAO UNIVE...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaN,nan,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,MR.RAM DEVIDAYAL HAS A MASTER'S DEGREE IN COMM...,NaT,U,2007-10-13,NaT,NaT,2010-03-31,0,0.0,1.0,1.0,0,0.0,0.0,0.0,535.0,0.0,0.0,170.0,0.0,0.0,170.0,535.0,0.0,0.0,535.0,170.0,0.0,0.0,170.0,535.0,0.0,0.0,535.0,2799
2,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027664,MR.,MIHIR,ARVIND,JOSHI,1977-06-12,M,NaN,NaN,NaN,nan,nan,"GRADUATE: ASSOCIATE MEMBER,INSTITUTE OF COMPAN...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"MR.MIHIR JOSHI, GVFL- NOMINEE DIRECTOR HOLDS D...",NaT,U,2007-08-23,NaT,NaT,2010-03-31,0,0.0,1.0,1.0,0,0.0,0.0,0.0,586.0,0.0,0.0,221.0,0.0,0.0,221.0,586.0,0.0,0.0,586.0,221.0,0.0,0.0,221.0,586.0,0.0,0.0,586.0,586
3,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00029006,MR.,ATIL,CHANDRESH,PARIKH,1977-09-04,M,INDIA,NaN,NaN,nan,nan,"GRADUATE: B.E.-CHEMICAL,GUJARAT UNIVERSITY,AHM...","POST GRADUATE: M.B.A.-FINANCE,CALIFORNIA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CORPORATE GOVERNANCE,F...",NaN,nan,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"MR. ATIL C. PARIKH, HOLDS A BACHELOR'S DEGREE ...",NaT,U,2001-01-29,NaT,NaT,2010-03-31,0,0.0,1.0,1.0,0,0.0,0.0,0.0,2983.0,0.0,0.0,2618.0,0.0,0.0,2618.0,2983.0,0.0,0.0,2983.0,2618.0,0.0,0.0,2618.0,2983.0,0.0,0.0,2983.0,2981
4,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027661,LATE MR.,CHANDRESH,SHANTILAL,PARIKH,1946-03-06,M,INDIA,NaN,NaN,nan,nan,POST GRADUATE: M.SC.-CHEMISTRY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CORPORATE GOVERNANCE,F...",NaN,nan,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"MR.CHANDRESH S. PARIKH, OUR CHAIRMAN AND MANAG...",NaT,U,1988-03-29,NaT,NaT,2010-03-31,0,0.0,1.0,1.0,0,0.0,0.0,0.0,7672.0,0.0,0.0,7307.0,0.0,0.0,7307.0,7672.0,0.0,0.0,7672.0,7307.0,0.0,0.0,7307.0,7672.0,0.0,0.0,7672.0,7667
...,...,...,...,.

## COMP COUNT CALCULATIONS

In [56]:
compCountColNum = [0,1,2,5,47,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44]
compCountCol = mainDirect2.columns[compCountColNum]
compCountCol

Index(['Symbol', 'Company', 'AsOnDate', 'Person Code', 'Indep',
       'Other Directorship 1', 'Other Directorship 2', 'Other Directorship 3',
       'Other Directorship 4', 'Other Directorship 5', 'Other Directorship 6',
       'Other Directorship 7', 'Other Directorship 8', 'Other Directorship 9',
       'Other Directorship 10', 'Other Directorship 11',
       'Other Directorship 12', 'Other Directorship 13',
       'Other Directorship 14', 'Other Directorship 15'],
      dtype='object')

In [57]:
compDirect = mainDirect2[compCountCol]
compDirect

,Symbol,Company,AsOnDate,Person Code,Indep,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15
0,20MICRONS,20 MICRONS LTD.,2009-03-31,P00027662,U,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,20MICRONS,20 MICRONS LTD.,2009-03-31,P00027633,U,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,20MICRONS,20 MICRONS LTD.,2009-03-31,P00027664,U,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,20MICRONS,20 MICRONS LTD.,2009-03-31,P00029006,U,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,20MICRONS,20 MICRONS LTD.,2009-03-31,P00027661,U,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258252,ZYLOG,ZYLOG SYSTEMS LTD.,2018-03-31,P00114429,U,ZYLOG SYSTEMS LTD.,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
258253,ZYLOG,ZYLOG SYSTEMS LTD.,2018-03-31,P00080494,Y,ZYLOG SYSTEMS LTD.,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
258254,ZYLOG,ZYLOG SYSTEMS LTD.,2018-03-31,P00067111,Y,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
258255,ZYLOG,ZYLOG SYSTEMS LTD.,2018-03-31,P00036956,Y,RAVIKUMAR DISTILLERIES LTD.,ZYLOG SYSTEMS LTD.,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


### 1 PAST DIRECTORSHIPS 1

In [58]:
# Per person, per AsOnDate, we get all the previous Directorships along with their Indep status

def PastDirect(frame):
    frameOut = pd.DataFrame(index = frame.index, columns = ['AsOnDate','Past Directorships']) # Output Frame
    
    for i in range(len(frame)):                                                               # iterating through the length of input frame
        Date = frame.iloc[i]['AsOnDate']                                                      # Date = AsOnDate for that iteration
        framePast = frame.loc[ frame['AsOnDate'] < Date , ['Symbol', 'Indep']]                # Creating a frame with date before current Date
        pastDirectors = list(zip(framePast['Symbol'], framePast['Indep']))                    # list of tuples of the company and indep status in the past_frame
        pastDirUnique = tuple(set(pastDirectors))                                             # converting the above to a set, to remove duplicates, and then a list for easier use

        frameOut.at[ frame.index[i], 'Past Directorships'] = pastDirUnique                    # putting the above data to the output frame
        frameOut.at[ frame.index[i], 'AsOnDate'] = Date

    return frameOut

In [59]:
pastDirect = compDirect.sort_values(by = ['Person Code', 'AsOnDate', 'Company']).groupby('Person Code').progress_apply(PastDirect)

100%|███████████████████████████████████████████████████████████████████████████| 35290/35290 [05:49<00:00, 100.87it/s]


In [60]:
pastDirect

AsOnDate Past Directorships
Person Code                                               
P00000005   5559    2006-03-31 00:00:00                 ()
            5570    2007-03-31 00:00:00   ((AEGISLOG, U),)
            5581    2008-03-31 00:00:00   ((AEGISLOG, U),)
            5591    2009-03-31 00:00:00   ((AEGISLOG, U),)
            5601    2010-03-31 00:00:00   ((AEGISLOG, U),)
...                                 ...                ...
P00123356   23607   2024-03-31 00:00:00                 ()
P00123357   23609   2024-03-31 00:00:00                 ()
P00123448   159306  2024-03-31 00:00:00                 ()
P00123948   70493   2024-03-31 00:00:00                 ()
P00123949   70488   2024-03-31 00:00:00                 ()

[258257 rows x 2 columns]

In [61]:
pastDirect2 = pastDirect.reset_index().drop( labels = ["level_1"], axis = 1 ).drop_duplicates().rename( {"Past Directorships":"AllPastDirect"}, axis = 1)

In [62]:
pastDirect2

,Person Code,AsOnDate,AllPastDirect
0,P00000005,2006-03-31 00:00:00,()
1,P00000005,2007-03-31 00:00:00,"((AEGISLOG, U),)"
2,P00000005,2008-03-31 00:00:00,"((AEGISLOG, U),)"
3,P00000005,2009-03-31 00:00:00,"((AEGISLOG, U),)"
4,P00000005,2010-03-31 00:00:00,"((AEGISLOG, U),)"
...,...,...,...
258252,P00123356,2024-03-31 00:00:00,()
258253,P00123357,2024-03-31 00:00:00,()
258254,P00123448,2024-03-31 00:00:00,()
258255,P00123948,2024-03-31 00:00:00,()


### 1 CURRENT DIRECTORSHIPS 1

In [63]:
otherDirectColNum = [30,31,32,33,34,35,36,37,38,39,40,41,42,43,44]
otherDirectCol = mainDirect2.columns[otherDirectColNum]
otherDirectCol

Index(['Other Directorship 1', 'Other Directorship 2', 'Other Directorship 3',
       'Other Directorship 4', 'Other Directorship 5', 'Other Directorship 6',
       'Other Directorship 7', 'Other Directorship 8', 'Other Directorship 9',
       'Other Directorship 10', 'Other Directorship 11',
       'Other Directorship 12', 'Other Directorship 13',
       'Other Directorship 14', 'Other Directorship 15'],
      dtype='object')

In [64]:
# A = Current Directorships in other records
# B = Current Directorships in other directorships columns



# A if A in B
def CurrentDirectAinB(frame, cols): 
    # Ensure cols is a list of column names and that they exist in the DataFrame
    if not all(col in frame.columns for col in cols):
        raise ValueError("One or more columns in 'cols' are not in the DataFrame")

    # Create unique list of tuples from 'Company' and 'Indep' in the current group
    comp_stat = list(set(zip(frame.Company, frame.Indep)))
    
    # Initialize the new DataFrame
    
    frameOut = pd.DataFrame(index=frame.index, columns=['Current Direct'])
    
    # Populate 'Current Direct' column
    for i in range(len(frame)):
        # Get the value for each column - All other directorships from other directorships columns
        row_values = [str(frame[col].iloc[i]) for col in cols]
        # Check if any company from comp_stat is in row_values - If companies within the frame are listed in other directorships columns
        current_direct = tuple(
            item for item in comp_stat if item[0] in row_values
        )
        frameOut.loc[frame.index[i], 'Current Direct'] = current_direct
    
    return frameOut

In [65]:
currentDirectAinB = compDirect.groupby(['Person Code', 'AsOnDate']).progress_apply(CurrentDirectAinB, cols=otherDirectCol)

100%|█████████████████████████████████████████████████████████████████████████| 202847/202847 [15:04<00:00, 224.20it/s]


In [66]:
currentDirectAinB

Current Direct
Person Code AsOnDate                                                
P00000005   2006-03-31 5559                                       ()
            2007-03-31 5570                                       ()
            2008-03-31 5581                                       ()
            2009-03-31 5591                                       ()
            2010-03-31 5601                                       ()
...                                                              ...
P00123356   2024-03-31 23607           ((AZAD ENGINEERING LTD., U),)
P00123357   2024-03-31 23609           ((AZAD ENGINEERING LTD., U),)
P00123448   2024-03-31 159306     ((NIIT LEARNING SYSTEMS LTD., U),)
P00123948   2024-03-31 70493   ((FLAIR WRITING INDUSTRIES LTD., U),)
P00123949   2024-03-31 70488   ((FLAIR WRITING INDUSTRIES LTD., U),)

[258257 rows x 1 columns]

In [67]:
noDataDates = ["2006-03-31", "2007-03-31", "2008-03-31", "2009-03-31", "2010-03-31", "2011-03-31", "2012-03-31"]
noDataDates = pd.to_datetime( noDataDates, format = "%Y-%m-%d")
                             
currentDirect2 = currentDirectAinB.reset_index( drop = False ).drop(["level_2"], axis = 1).drop_duplicates().reset_index( drop = True )
currentDirect3 = currentDirect2.loc[~currentDirect2.AsOnDate.isin(noDataDates)]
currentDirect3

,Person Code,AsOnDate,Current Direct
7,P00000005,2013-03-31,"((AEGIS LOGISTICS LTD., N),)"
8,P00000005,2014-03-31,"((AEGIS LOGISTICS LTD., N),)"
9,P00000005,2015-03-31,"((AEGIS LOGISTICS LTD., N),)"
10,P00000005,2016-03-31,"((AEGIS LOGISTICS LTD., N),)"
11,P00000005,2017-03-31,"((AEGIS LOGISTICS LTD., N),)"
...,...,...,...
202842,P00123356,2024-03-31,"((AZAD ENGINEERING LTD., U),)"
202843,P00123357,2024-03-31,"((AZAD ENGINEERING LTD., U),)"
202844,P00123448,2024-03-31,"((NIIT LEARNING SYSTEMS LTD., U),)"
202845,P00123948,2024-03-31,"((FLAIR WRITING INDUSTRIES LTD., U),)"


In [68]:
# A = Current Directorships in other records 

def CurrentDirectA(frame):
    cols=otherDirectCol

    # Create unique list of tuples from 'Company' and 'Indep' in the current group
    comp_stat = tuple(set(zip(frame.Company, frame.Indep)))

    return comp_stat

In [69]:
currentDirectA = compDirect.groupby(['Person Code', 'AsOnDate']).progress_apply(CurrentDirectA)

100%|███████████████████████████████████████████████████████████████████████| 202847/202847 [00:18<00:00, 11022.59it/s]


In [70]:
currentDirectA

Person Code  AsOnDate  
P00000005    2006-03-31             ((AEGIS LOGISTICS LTD., U),)
             2007-03-31             ((AEGIS LOGISTICS LTD., U),)
             2008-03-31             ((AEGIS LOGISTICS LTD., U),)
             2009-03-31             ((AEGIS LOGISTICS LTD., U),)
             2010-03-31             ((AEGIS LOGISTICS LTD., U),)
                                           ...                  
P00123356    2024-03-31            ((AZAD ENGINEERING LTD., U),)
P00123357    2024-03-31            ((AZAD ENGINEERING LTD., U),)
P00123448    2024-03-31       ((NIIT LEARNING SYSTEMS LTD., U),)
P00123948    2024-03-31    ((FLAIR WRITING INDUSTRIES LTD., U),)
P00123949    2024-03-31    ((FLAIR WRITING INDUSTRIES LTD., U),)
Length: 202847, dtype: object

In [71]:
currentDirectA2 = currentDirectA.to_frame().reset_index( drop = False ).drop_duplicates().rename({0:"CurrDirectA"}, axis = 1).reset_index( drop = True )
currentDirectA2

,Person Code,AsOnDate,CurrDirectA
0,P00000005,2006-03-31,"((AEGIS LOGISTICS LTD., U),)"
1,P00000005,2007-03-31,"((AEGIS LOGISTICS LTD., U),)"
2,P00000005,2008-03-31,"((AEGIS LOGISTICS LTD., U),)"
3,P00000005,2009-03-31,"((AEGIS LOGISTICS LTD., U),)"
4,P00000005,2010-03-31,"((AEGIS LOGISTICS LTD., U),)"
...,...,...,...
202842,P00123356,2024-03-31,"((AZAD ENGINEERING LTD., U),)"
202843,P00123357,2024-03-31,"((AZAD ENGINEERING LTD., U),)"
202844,P00123448,2024-03-31,"((NIIT LEARNING SYSTEMS LTD., U),)"
202845,P00123948,2024-03-31,"((FLAIR WRITING INDUSTRIES LTD., U),)"


In [72]:
# comparison shows that there is a mismatch, which means that other directorships and current directorships may not be same, Ill check it outside in excel
# currentDirect3.to_csv("Check1.csv")
# currentDirectCOMPARE3.to_csv("Check2.csv")

In [73]:
# A + B = Current Directorships in directorships rows + Current Directorships in other directorships columns

def CurrentDirectAB(frame):
    cols=otherDirectCol

    # Create unique list of tuples from 'Company' and 'Indep' in the current group
    comp_stat = list(set(zip(frame.Company, frame.Indep)))

    for i in range(len(frame)):
        # Create a list of companies in other direct columns
        other_direct = set(str(frame[col].iloc[i]) for col in cols)
        other_direct2 = tuple(x for x in other_direct if x==x)
    
    other_direct3 = list(set(zip([x for x in other_direct2], "U")))
    
    current_direct = tuple(comp_stat + other_direct3)

    current_direct2 = tuple(x for x in current_direct if x[0] != "nan" )
    
    return current_direct2

In [74]:
currentDirectAB = compDirect.groupby(['Person Code', 'AsOnDate']).progress_apply(CurrentDirectAB)

100%|████████████████████████████████████████████████████████████████████████| 202847/202847 [01:49<00:00, 1857.28it/s]


In [75]:
currentDirectAB

Person Code  AsOnDate  
P00000005    2006-03-31                         ((AEGIS LOGISTICS LTD., U),)
             2007-03-31                         ((AEGIS LOGISTICS LTD., U),)
             2008-03-31                         ((AEGIS LOGISTICS LTD., U),)
             2009-03-31                         ((AEGIS LOGISTICS LTD., U),)
             2010-03-31                         ((AEGIS LOGISTICS LTD., U),)
                                                 ...                        
P00123356    2024-03-31    ((AZAD ENGINEERING LTD., U), (AZAD ENGINEERING...
P00123357    2024-03-31    ((AZAD ENGINEERING LTD., U), (AZAD ENGINEERING...
P00123448    2024-03-31                   ((NIIT LEARNING SYSTEMS LTD., U),)
P00123948    2024-03-31                ((FLAIR WRITING INDUSTRIES LTD., U),)
P00123949    2024-03-31                ((FLAIR WRITING INDUSTRIES LTD., U),)
Length: 202847, dtype: object

In [76]:
# currentDirectFINAL.to_csv("Check3.csv")

In [77]:
currentDirectAB2 = currentDirectAB.to_frame().reset_index( drop = False ).drop_duplicates().rename({0:"CurrDirectAB"}, axis = 1).reset_index( drop = True )

In [78]:
currentDirectAB2

,Person Code,AsOnDate,CurrDirectAB
0,P00000005,2006-03-31,"((AEGIS LOGISTICS LTD., U),)"
1,P00000005,2007-03-31,"((AEGIS LOGISTICS LTD., U),)"
2,P00000005,2008-03-31,"((AEGIS LOGISTICS LTD., U),)"
3,P00000005,2009-03-31,"((AEGIS LOGISTICS LTD., U),)"
4,P00000005,2010-03-31,"((AEGIS LOGISTICS LTD., U),)"
...,...,...,...
202842,P00123356,2024-03-31,"((AZAD ENGINEERING LTD., U), (AZAD ENGINEERING..."
202843,P00123357,2024-03-31,"((AZAD ENGINEERING LTD., U), (AZAD ENGINEERING..."
202844,P00123448,2024-03-31,"((NIIT LEARNING SYSTEMS LTD., U),)"
202845,P00123948,2024-03-31,"((FLAIR WRITING INDUSTRIES LTD., U),)"


In [79]:
compSymbol = mainDirect2[["Company", "Symbol"]].copy().drop_duplicates()
compSymboldict = dict(zip(compSymbol["Company"], compSymbol["Symbol"]))

currentDirectAB2["CurrDirectAB_Final"] = currentDirectAB2["CurrDirectAB"].progress_apply(lambda list: set((compSymboldict[x[0]],x[1]) for x in list))
currentDirectA2["CurrDirectA_Final"] = currentDirectA2["CurrDirectA"].progress_apply(lambda list: set((compSymboldict[x[0]],x[1]) for x in list))

100%|██████████████████████████████████████████████████████████████████████| 202847/202847 [00:00<00:00, 434240.83it/s]


### 1 COMP COUNT 1

In [80]:
pastDirect2["AsOnDate"] = pd.to_datetime(pastDirect2["AsOnDate"], format = "%Y-%m-%d")

# Creating a comp count only variables dataframe
compCountPre = pastDirect2.merge( currentDirectA2[['Person Code', 'AsOnDate', 'CurrDirectA_Final']], on = ['Person Code', 'AsOnDate'] )
compCountPre2 = compCountPre.merge( currentDirectAB2[['Person Code', 'AsOnDate', 'CurrDirectAB_Final']], on = ['Person Code', 'AsOnDate'] )
compCountPre2 = compCountPre2.rename({"CurrDirectA_Final":"CurrDirectA", "CurrDirectAB_Final":"CurrDirectAB"}, axis = 1)

In [81]:
# creating a tuple of Past Directorships

compCountPre2['PastSiezedDirectA'] = compCountPre2.progress_apply(
    lambda row: tuple(x for x in row['AllPastDirect'] if x not in row['CurrDirectA']), axis = 1)

compCountPre2['PastSiezedDirectAB'] = compCountPre2.progress_apply(
    lambda row: tuple(x for x in row['AllPastDirect'] if x not in row['CurrDirectAB']), axis = 1)

100%|███████████████████████████████████████████████████████████████████████| 202847/202847 [00:02<00:00, 92488.36it/s]


In [82]:
compCount = compCountPre2.copy()

In [83]:
# Count Other Past Directorships A

compCount['CountOtherPastUnclearA'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['PastSiezedDirectA'] if x[1]=='U'), axis = 1)

compCount['CountOtherPastIndepA'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['PastSiezedDirectA'] if x[1]=='Y'), axis = 1)

compCount['CountOtherPastNonIndepA'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['PastSiezedDirectA'] if x[1]=='N'), axis = 1)

compCount['CompCountOtherPastTotalA'] = compCount['CountOtherPastUnclearA']+compCount['CountOtherPastIndepA']+compCount['CountOtherPastNonIndepA']

# Count Other Past Directorships AB

compCount['CountOtherPastUnclearAB'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['PastSiezedDirectAB'] if x[1]=='U'), axis = 1)

compCount['CountOtherPastIndepAB'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['PastSiezedDirectAB'] if x[1]=='Y'), axis = 1)

compCount['CountOtherPastNonIndepAB'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['PastSiezedDirectAB'] if x[1]=='N'), axis = 1)

compCount['CompCountOtherPastTotalAB'] = compCount['CountOtherPastUnclearAB']+compCount['CountOtherPastIndepAB']+compCount['CountOtherPastNonIndepAB']

100%|██████████████████████████████████████████████████████████████████████| 202847/202847 [00:01<00:00, 125689.25it/s]


In [84]:
# Count Current Directorships A

compCount['CountCurrUnclearA'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['CurrDirectA'] if x[1]=='U'), axis = 1)

compCount['CountCurrIndepA'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['CurrDirectA'] if x[1]=='Y'), axis = 1)

compCount['CountCurrNonIndepA'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['CurrDirectA'] if x[1]=='N'), axis = 1)

compCount['CompCountCurrTotalA'] = compCount['CountCurrUnclearA']+compCount['CountCurrIndepA']+compCount['CountCurrNonIndepA']

# Count Current Directorships AB

compCount['CountCurrUnclearAB'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['CurrDirectAB'] if x[1]=='U'), axis = 1)

compCount['CountCurrIndepAB'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['CurrDirectAB'] if x[1]=='Y'), axis = 1)

compCount['CountCurrNonIndepAB'] = compCount.progress_apply(
    lambda row: sum(1 for x in row['CurrDirectAB'] if x[1]=='N'), axis = 1)

compCount['CompCountCurrTotalAB'] = compCount['CountCurrUnclearAB']+compCount['CountCurrIndepAB']+compCount['CountCurrNonIndepAB']

100%|██████████████████████████████████████████████████████████████████████| 202847/202847 [00:01<00:00, 121076.36it/s]


In [85]:
compCount

,Person Code,AsOnDate,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB
0,P00000005,2006-03-31,(),"{(AEGISLOG, U)}","{(AEGISLOG, U)}",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1
1,P00000005,2007-03-31,"((AEGISLOG, U),)","{(AEGISLOG, U)}","{(AEGISLOG, U)}",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1
2,P00000005,2008-03-31,"((AEGISLOG, U),)","{(AEGISLOG, U)}","{(AEGISLOG, U)}",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1
3,P00000005,2009-03-31,"((AEGISLOG, U),)","{(AEGISLOG, U)}","{(AEGISLOG, U)}",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1
4,P00000005,2010-03-31,"((AEGISLOG, U),)","{(AEGISLOG, U)}","{(AEGISLOG, U)}",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202842,P00123356,2024-03-31,(),"{(AZAD, U)}","{(AZAD, U)}",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1
202843,P00123357,2024-03-31,(),"{(AZAD, U)}","{(AZAD, U)}",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1
202844,P00123448,2024-03-31,(),"{(NIITMTS, U)}","{(NIITMTS, U)}",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1
202845,P00123948,2024-03-31,(),"{(FLAIR, U)}","{(FLAIR, U)}",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1


### 2 MERGE WITH MAIN 3

In [86]:
mainDirect3 = mainDirect2.merge( compCount, on = ["Person Code", "AsOnDate"] )
mainDirect3

,Symbol,Company,AsOnDate,AsOnYear,ISIN,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Member of Civil Services,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Occupation,Cessation Reason,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Brief Profile,Tenure Valid till,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,AppointDummy,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal,CloBalTotalXP,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB
0,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027662,MR.,DIPTIMOY,DINESHCHANDRA,BHATTACHARJEE,1955-12-09,M,NaN,NaN,NaN,nan,nan,GRADUATE: B.SC.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOMINATION ENDED,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"MR.D.D.BHATTACHARJEE, IDBI-NOMINEE DIRECTOR HO...",NaT,U,2005-04-29,2009-03-02,NaT,2010-03-31,1,0.0,1.0,1.0,0,0.0,0.0,0.0,1403.0,0.0,0.0,1038.0,0.0,0.0,1038.0,1403.0,0.0,0.0,1403.0,1038.0,0.0,0.0,1038.0,1403.0,0.0,0.0,1403.0,1402,(),"{(20MICRONS, U)}","{(20MICRONS, U)}",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1
1,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027633,MR.,RAMKISHAN,AMIRCHAND,DEVIDAYAL,1950-12-08,M,INDIA,NONE,NaN,nan,nan,POST GRADUATE: M.B.A.,"POST GRADUATE: M.COM.,MAHARAJA SAYAJIRAO UNIVE...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaN,nan,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,MR.RAM DEVIDAYAL HAS A MASTER'S DEGREE IN COMM...,NaT,U,2007-10-13,NaT,NaT,2010-03-31,0,0.0,1.0,1.0,0,0.0,0.0,0.0,535.0,0.0,0.0,170.0,0.0,0.0,170.0,535.0,0.0,0.0,535.0,170.0,0.0,0.0,170.0,535.0,0.0,0.0,535.0,2799,"((BANCOINDIA, U),)","{(20MICRONS, U), (MUNJALAU, U), (BANCOINDIA, U)}","{(20MICRONS, U), (MUNJALAU, U), (BANCOINDIA, U)}",(),(),0,0,0,0,0,0,0,0,3,0,0,3,3,0,0,3
2,BANCOINDIA,BANCO PRODUCTS (INDIA) LTD.,2009-03-31,2009,INE213C01025,P00027633,MR.,RAMKISHAN,AMIRCHAND,DEVIDAYAL,1950-12-08,M,INDIA,NONE,NaN,nan,nan,POST GRADUATE: M.B.A.,"POST GRADUATE: M.COM.,MAHARAJA SAYAJIRAO UNIVE...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",NaN,nan,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,MR. RAM DEVIDAYAL HOLDS MASTER DEGREE IN COMME...,NaT,U,2001-07-30,NaT,2008-03-31,2010-03-31,0,0.0,0.0,1.0,0,0.0,0.0,0.0,365.0,0.0,0.0,2436.0,0.0,0.0,2436.0,2801.0,0.0,0.0,2801.0,2436.0,0.0,0.0,2436.0,2801.0,0.0,0.0,2801.0,2799,"((BANCOINDIA, U),)","{(20MICRONS, U), (MUNJALAU, U), (BANCOINDIA, U)}","{(20MICRONS, U), (MUNJALAU, U), (BANCOINDIA, U)}",(),(),0,0,0,0,0,0,0,0,3,0,0,3,3,0,0,3
3,MUNJALAU,MUNJAL AUTO INDUSTRIES LTD.,2009-03-31,2009,INE672B01032,P00027633,MR.,RAMKISHAN,AMIRCHAND,DEVIDAYAL,1950-12-08,M,INDIA,NONE,NaN,nan,nan,POST GRADUATE: M.B.A.,"POST GRADUATE: M.COM.,MAHARAJA SAYAJIRAO UNIVE...",NaN,NaN,NaN,NaN,NaN

## 3 DIRECTOR ROOKIE CLASSIFICATION AND FURTHER VARIABLES 3

In [87]:
mainDirect3.loc[ mainDirect3['CloBalTotalXP'] < 365*3, 'Rookie'] = 'Y'
mainDirect3.loc[ mainDirect3['CloBalTotalXP'] >= 365*3, 'Rookie'] = 'N'

In [88]:
mainDirect3['IsIndep'] = np.where( mainDirect3['Indep'] == 'Y', 1, 0)
mainDirect3['IsNonIndep'] =  np.where( mainDirect3['Indep'] == 'Y', 0, 1)

mainDirect3['IsRookie'] = np.where( mainDirect3['Rookie'] == 'Y', 1, 0)
mainDirect3['IsNonRookie'] = np.where( mainDirect3['Rookie'] == 'Y', 0, 1)

In [89]:
mainDirect3['Position on Board'].value_counts()

Position on Board
NON-EXECUTIVE DIRECTOR                                                                                              99028
nan                                                                                                                 79856
EXECUTIVE DIRECTOR                                                                                                  10299
NON-EXECUTIVE DIRECTOR, PROMOTER-DIRECTOR                                                                            6362
EXECUTIVE DIRECTOR, PROMOTER-DIRECTOR                                                                                6157
                                                                                                                    ...  
NOMINEE DIRECTOR OF ASSET RECONSTRUCTION CO.(INDIA) LTD., EXECUTIVE DIRECTOR                                            1
NOMINEE DIRECTOR OF SUZUKI MOTOR CORP., JT.MANAGING DIRECTOR-ENGINEERING & QUALITY ASSURANCE, EXECUTIVE DIRECTOR        1
NOMINE

In [90]:
mainDirect3['Position on Board'] = mainDirect3['Position on Board'].astype(str)
mainDirect3['IsCeoMDPosition'] = np.where( 
    mainDirect3['Position on Board'].str.contains(('CEO|CHIEF EXECUTIVE OFFICER|MANAGING DIRECTOR|MD'), case = False, na = False ) , 1,0)

In [91]:
mainDirect3['IsCeoMDPosition'].value_counts()

IsCeoMDPosition
0    235902
1     22355
Name: count, dtype: int64

In [92]:
mainDirect3['IsChairmanPosition'] = np.where( 
    mainDirect3['Position on Board'].str.contains(('CHAIRMAN|CHAIR PERSON|CHAIRPERSON|CHAIR'), case = False, na = False ) , 1,0)

mainDirect3['IsChairmanPosition'] = np.where( 
    mainDirect3['Position on Board'].str.contains(('DY. CHAIRPERSON|DY. CHAIRMAN|DY.CHAIRPERSON|VICE CHAIRPERSON|VICE-CHAIRPERSON|VICE -CHAIRPERSON|VICE- CHAIRPERSON|VICE - CHAIRPERSON|VICE CHAIRMAN|VICE-CHAIRMAN|VICE -CHAIRMAN|VICE- CHAIRMAN|VICE - CHAIRMAN'), case = False, na = False ) , 0, mainDirect3['IsChairmanPosition'])

In [93]:
mainDirect3['IsChairmanPosition'].value_counts()

IsChairmanPosition
0    237576
1     20681
Name: count, dtype: int64

In [94]:
mainDirect3['Occupation'].value_counts()

Occupation
RETIRED                                                                                                                 10226
BUSINESS                                                                                                                 3920
CONSULTANT                                                                                                               1588
INDUSTRIALIST                                                                                                            1516
SERVICE                                                                                                                  1214
                                                                                                                        ...  
MANAGING DIRECTOR & EXECUTIVE DIRECTOR, UNIVASTU INDIA LTD.                                                                 1
PRESIDENT, BHARATIYA JANATA PARTY, KERALA                                                                  

In [95]:
mainDirect3['Occupation'] = mainDirect3['Occupation'].astype(str)
mainDirect3['IsCeoMDOccupation'] = np.where( 
    mainDirect3['Occupation'].str.contains(('CEO|CHIEF EXECUTIVE OFFICER|MANAGING DIRECTOR|MD'), case = False, na = False ) , 1,0)

In [96]:
mainDirect3['IsCeoMDOccupation'].value_counts()

IsCeoMDOccupation
0    221711
1     36546
Name: count, dtype: int64

In [97]:
mainDirect3['IsChairmanOccupation'] = np.where( 
    mainDirect3['Occupation'].str.contains(('CHAIRMAN|CHAIR PERSON|CHAIRPERSON|CHAIR'), case = False, na = False ) , 1,0)

mainDirect3['IsChairmanOccupation'] = np.where( 
    mainDirect3['Occupation'].str.contains(('DY. CHAIRPERSON|DY. CHAIRMAN|DY.CHAIRPERSON|VICE CHAIRPERSON|VICE-CHAIRPERSON|VICE -CHAIRPERSON|VICE- CHAIRPERSON|VICE - CHAIRPERSON|VICE CHAIRMAN|VICE-CHAIRMAN|VICE -CHAIRMAN|VICE- CHAIRMAN|VICE - CHAIRMAN'), case = False, na = False ) , 0, mainDirect3['IsChairmanOccupation'])

In [98]:
mainDirect3['IsChairmanOccupation'].value_counts()

IsChairmanOccupation
0    239012
1     19245
Name: count, dtype: int64

In [99]:
mainDirect3['IsCeoMD'] = np.where( (mainDirect3['IsCeoMDPosition'] == 1) | (mainDirect3['IsCeoMDOccupation'] == 1) , 1, 0)
mainDirect3['IsCeoMD'].value_counts()

IsCeoMD
0    219213
1     39044
Name: count, dtype: int64

In [100]:
mainDirect3['IsChairman'] = np.where( (mainDirect3['IsChairmanPosition'] == 1) | (mainDirect3['IsChairmanOccupation'] == 1) , 1, 0)
mainDirect3['IsChairman'].value_counts()

IsChairman
0    230869
1     27388
Name: count, dtype: int64

In [101]:
mainDirect3['IsPromoterClassification'] = np.where( mainDirect3['Promoter Director (Yes/No)'] == 'YES', 1, 0)
mainDirect3['IsPromoterClassification'].value_counts()

IsPromoterClassification
0    222809
1     35448
Name: count, dtype: int64

In [102]:
mainDirect3['IsPromoterBoard'] = np.where(
    mainDirect3['Position on Board'].str.contains( ('PROMOTER'), case = False, na = False), 1, 0)
mainDirect3['IsPromoterBoard'].value_counts()

IsPromoterBoard
0    223736
1     34521
Name: count, dtype: int64

In [103]:
mainDirect3['IsPromoter'] = np.where( (mainDirect3['IsPromoterClassification'] == 1) | (mainDirect3['IsPromoterBoard'] == 1) , 1, 0)
mainDirect3['IsPromoter'].value_counts()

IsPromoter
0    220782
1     37475
Name: count, dtype: int64

In [104]:
mainDirect3['IsDualityChairmanMD'] = np.where( (mainDirect3['IsCeoMD'] == 1) & (mainDirect3['IsChairman'] == 1) , 1, 0)
mainDirect3['IsDualityChairmanMD'].value_counts()

IsDualityChairmanMD
0    245535
1     12722
Name: count, dtype: int64

In [105]:
mainDirect3['IsFamilyManager'] = np.where( (mainDirect3['IsCeoMD'] == 1) & (mainDirect3['IsPromoter'] == 1) , 1, 0)
mainDirect3['IsFamilyManager'].value_counts()

IsFamilyManager
0    240789
1     17468
Name: count, dtype: int64

In [106]:
mainDirect3['IsFamilyChairman'] = np.where( (mainDirect3['IsPromoter'] == 1) & (mainDirect3['IsChairman'] == 1) , 1, 0)
mainDirect3['IsFamilyChairman'].value_counts()

IsFamilyChairman
0    245104
1     13153
Name: count, dtype: int64

In [107]:
mainDirect3['IsFamilyChairmanAndCEO'] = np.where( (mainDirect3['IsFamilyChairman'] == 1) & (mainDirect3['IsFamilyManager'] == 1) , 1, 0)
mainDirect3['IsFamilyChairmanAndCEO'].value_counts()

IsFamilyChairmanAndCEO
0    250560
1      7697
Name: count, dtype: int64

In [108]:
mainDirect3['IsRookieIndep'] = np.where( (mainDirect3['IsRookie'] == 1) & (mainDirect3['IsIndep'] == 1) , 1, 0)
mainDirect3['IsRookieIndep'].value_counts()

IsRookieIndep
0    236235
1     22022
Name: count, dtype: int64

In [109]:
mainDirect3['IsRookieNonIndep'] = np.where( (mainDirect3['IsRookie'] == 1) & (mainDirect3['IsIndep'] == 0) , 1, 0)
mainDirect3['IsRookieNonIndep'].value_counts()

IsRookieNonIndep
0    217768
1     40489
Name: count, dtype: int64

In [110]:
mainDirect3['IsNonRookieIndep'] = np.where( (mainDirect3['IsRookie'] == 0) & (mainDirect3['IsIndep'] == 1) , 1, 0)
mainDirect3['IsNonRookieIndep'].value_counts()

IsNonRookieIndep
0    186343
1     71914
Name: count, dtype: int64

In [111]:
mainDirect3['IsNonRookieNonIndep'] = np.where( (mainDirect3['IsRookie'] == 0) & (mainDirect3['IsIndep'] == 0) , 1, 0)
mainDirect3['IsNonRookieNonIndep'].value_counts()

IsNonRookieNonIndep
0    134425
1    123832
Name: count, dtype: int64

In [112]:
mainDirect3['IsFemale'] = np.where( (mainDirect3['Gender'] == 'F'), 1,0)
mainDirect3['IsFemale'].value_counts()

IsFemale
0    227430
1     30827
Name: count, dtype: int64

In [113]:
now = pd.Timestamp('now')

mainDirect3['Date of Birth'] = pd.to_datetime(mainDirect3['Date of Birth'], format = '%Y-%m-%d')

mainDirect3['Age'] = np.floor((now - mainDirect3['Date of Birth']).dt.days / 365).astype('Int64')

In [114]:
mainDirect3['TenureInYearsinCompIndep'] = mainDirect3['CompCloBalIndep']/365
mainDirect3['TenureInYearsinCompTotal'] = mainDirect3['CompCloBalTotal']/365

## 3 Retirement and First Term 4

In [115]:
mainDirect3['IsFirstTerm'] = np.where((mainDirect3['CompCloBalTotal'] < 365*5), 1, 0)
mainDirect3['IsFirstTermIndep'] = np.where( (mainDirect3['IsIndep'] == 1) & (mainDirect3['CompCloBalTotal'] < 365*5), 1, 0)

mainDirect3['IsZeroYear'] = np.where((mainDirect3['CloBalTotalXP'] == 0), 1, 0)
mainDirect3['IsZeroYearIndep'] = np.where( (mainDirect3['IsIndep'] == 1) & (mainDirect3['CloBalTotalXP'] == 0), 1, 0)

mainDirect3['IsOneYear'] = np.where((mainDirect3['CloBalTotalXP'] < 365*1), 1, 0)
mainDirect3['IsOneYearIndep'] = np.where( (mainDirect3['IsIndep'] == 1) & (mainDirect3['CloBalTotalXP'] < 365*1), 1, 0)

mainDirect3['IsTwoYear'] = np.where((mainDirect3['CloBalTotalXP'] < 365*2), 1, 0)
mainDirect3['IsTwoYearIndep'] = np.where( (mainDirect3['IsIndep'] == 1) & (mainDirect3['CloBalTotalXP'] < 365*2), 1, 0)

mainDirect3['IsThreeYear'] = np.where((mainDirect3['CloBalTotalXP'] < 365*3), 1, 0)
mainDirect3['IsThreeYearIndep'] = np.where( (mainDirect3['IsIndep'] == 1) & (mainDirect3['CloBalTotalXP'] < 365*3), 1, 0)

mainDirect3['IsRetires5y'] = np.where(((mainDirect3['TermCloBalTotal'] <= 365*5) & (mainDirect3['TermCloBalTotal'] > 365*4))& 
                                      (mainDirect3['TermNumber'] == 1), 1, 0)
mainDirect3['IsTermLimitRetirement'] = np.where(((mainDirect3['TermCloBalTotal'] <= 365*5) & 
                                                 (mainDirect3['TermCloBalTotal'] > 365*4))& (mainDirect3['TermNumber'] == 2), 1, 0)
mainDirect3['IsDefaultTerm'] = np.where((mainDirect3['CompOpBalTotal'] > 365*10), 1, 0)

mainDirect3['IsBusy'] = np.where( mainDirect3['CompCountCurrTotalA'] > 2, 1,0)

mainDirect3['IsTurnOver'] = np.where(mainDirect3['CessationDummy'] == 1, 1, 0)

mainDirect3['HasRetires5y'] = np.where((mainDirect3['IsRetires5y'] == 1) & (mainDirect3['CessationDummy'] == 1), 1, 0)

mainDirect3['HasTermLimitRetirement'] = np.where((mainDirect3['IsTermLimitRetirement'] == 1) & (mainDirect3['CessationDummy'] == 1), 1, 0)

In [116]:
# def Retirement(frame):

# #aight shiv, get only the year from the date store it as integer. and check what is the max year(int) and put turnover =1 for the previous year or year-1
# #call apply by grouping person code and company

# #check if year+1 == exists at year where isretire5y = 1, if not, put hasretires5 at year = 1
# #check if year+1 == exists at year where IsTermLimitRetirement = 1, if not, put hasTermLimitRetirement at year = 1

#     years = frame['AsOnYear'].astype(int).tolist()

#     def Conditions(row):
#         turnover = 0
#         HasRetires5y = 0
#         HasTermLimitRetirement = 0
#         year = int(row['AsOnYear'])

#         if (year+1 not in years):
#             turnover = 1
#             if (row['IsRetires5y'] == 1):
#                 HasRetires5y = 1
#             if (row['IsTermLimitRetirement'] == 1):
#                 HasTermLimitRetirement = 1

#         return turnover, HasRetires5y, HasTermLimitRetirement

#     frame['IsTurnOver'], frame['HasRetires5y'], frame['HasTermLimitRetirement'] = zip(*frame.apply(Conditions, axis = 1))
#     return frame

# retire = mainDirect3.loc[ :, ['Person Code', 'Symbol', 'AsOnYear', 'IsRetires5y', 'IsTermLimitRetirement']]
# retire2 = retire.groupby(['Person Code', 'Symbol']).progress_apply(Retirement)
# retire2
# retire3 = retire2.reset_index( drop = True)

In [117]:
mainDirect4 = mainDirect3.copy()

In [118]:
mainDirect4['IsTurnOverIndep'] = np.where((mainDirect4['IsTurnOver'] == 1) & (mainDirect4['IsIndep'] == 1), 1, 0)

In [119]:
mainDirect4

,Symbol,Company,AsOnDate,AsOnYear,ISIN,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Member of Civil Services,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Occupation,Cessation Reason,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Brief Profile,Tenure Valid till,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,AppointDummy,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal,CloBalTotalXP,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMDPosition,IsChairmanPosition,IsCeoMDOccupation,IsChairmanOccupation,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManager,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsinCompIndep,TenureInYearsinCompTotal,IsFirstTerm,IsFirstTermIndep,IsZeroYear,IsZeroYearIndep,IsOneYear,IsOneYearIndep,IsTwoYear,IsTwoYearIndep,IsThreeYear,IsThreeYearIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,IsBusy,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep
0,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027662,MR.,DIPTIMOY,DINESHCHANDRA,BHATTACHARJEE,1955-12-09,M,NaN,NaN,NaN,nan,nan,GRADUATE: B.SC.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NOMINATION ENDED,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"MR.D.D.BHATTACHARJEE, IDBI-NOMINEE DIRECTOR HO...",NaT,U,2005-04-29,2009-03-02,NaT,2010-03-31,1,0.0,1.0,1.0,0,0.0,0.0,0.0,1403.0,0.0,0.0,1038.0,0.0,0.0,1038.0,1403.0,0.0,0.0,1403.0,1038.0,0.0,0.0,1038.0,1403.0,0.0,0.0,1403.0,1402,(),"{(20MICRONS, U)}","{(20MICRONS, U)}",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,69,0.00000,3.843836,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027633,MR.,RAMKISHAN,AMIRCHAND,DEVIDAYAL,1950-12-08,M,INDIA,NONE,NaN,nan,nan,POST GRADUATE: M.B.A.,"POST GRADUATE: M.COM.,MAHARAJA SAYAJIRAO UNIVE...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISCRETIONARY...",nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,MR.RAM DEVIDAYAL HAS A MASTER'S DEGREE IN COMM...,NaT,U,2007-10-13,NaT,NaT,2010-03-31,0,0.0,1.0,1.0,0,0.0,0.0,0.0,535.0,0.0,0.0,170.0,0.0,0.0,170.0,535.0,0.0,0.0,535.0,170.0,0.0,0.0,170.0,535.0,0.0,0.0,535.0,2799,"((BANCOINDIA, U),)","{(20MICRONS, U), (MUNJALAU, U), (BANCOINDIA, U)}","{(20MICRONS, U), (MUNJALAU, U), (BANCOINDIA, U)}",(),(),0,0,0,0,0,0,0,0,3,0,0,3,3,0,0,3,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,74,0.00000,1.465753,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,BANCOINDIA,BANCO PRODUCTS (INDIA) LTD.,2009-03-31,2009,INE213C01025,P00027633,MR.,RAMKISHAN,AMIRCHAND,DEVIDAYAL,1950-12-08,M,INDIA,NONE,NaN,nan,nan,

## 4 Past Experience, Skillsets, Education 5

In [120]:
indicators = pd.read_csv(rf"{supporting_folder_path}\Prowess IQ - Indicators 2.txt", sep="|", dtype = {"NIC code":str})
indicators

,Company Name,Prowess company code,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,NSE symbol,Head office address
0,'K' Steamship Agencies Pvt. Ltd.,3,Private Ltd.,10203010000,1971.0,Diversified,1.100000e+14,Diversified,34,20102000000,Private (Indian),Between 1951 and 1971,NaN,NaN
1,'X'Clusive Business Centre Pvt. Ltd.,307865,Private Ltd.,10203010000,1996.0,Diversified financial services,1.022000e+14,"Other financial service activities, except ins...",649,20102000000,Private (Indian),Between 1991 and 2013,NaN,NaN
2,1 To 1 Help.Net Pvt. Ltd.,591675,Private Ltd.,10203010000,2001.0,Business services & consultancy,1.010415e+14,Management consultancy activities,70200,20102000000,Private (Indian),Between 1991 and 2013,NaN,NaN
3,10C India Internet Pvt. Ltd.,556976,Private Ltd.,10203010000,2012.0,Telecommunication services,1.010406e+14,Other telecommunications activities,61900,20102000000,Private (Indian),Between 1991 and 2013,NaN,NaN
4,10I Commerce Services Pvt. Ltd.,560502,Private Ltd.,10203010000,2015.0,Wholesale trading,1.010404e+14,"Wholesale of radio, television and other consu...",46522,20102000000,Private (Indian),After 2014,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55316,Zylog Systems Ltd.,275793,Public Ltd.,10203020000,1995.0,Computer software,1.010408e+14,Providing software support and maintenance to ...,62013,20102000000,Private (Indian),Between 1991 and 2013,ZYLOG,NaN
55317,Zyma Laboratories Ltd. [Merged],275794,Public Ltd.,10203020000,1970.0,Other fund based financial services,1.020400e+14,"Trusts, funds and other financial vehicles",64300,20102000000,Private (Indian),Between 1951 and 1971,NaN,NaN
55318,Zyphar'S Pharmaceutics Pvt. Ltd.,565342,Private Ltd.,10203010000,2006.0,Wholesale trading,1.010404e+14,Wholesale of pharmaceutical and medical goods,46497,20102000000,Private (Indian),Between 1991 and 2013,NaN,NaN
55319,Zytel Agencies Ltd.,275795,Public Ltd.,10203020000,1984.0,Other fund based financial services,1.020400e+14,"Trusts, funds and other financial vehicles",64300,20102000000,Private (Indian),Between 1972 and 1985,NaN,NaN


In [121]:
indicator_col = ["NSE symbol", "NIC code", "Entity type", "Ownership group", "Prowess company code", "Company Name"]
mainDirect5 = mainDirect4.merge(indicators[indicator_col], left_on = "Symbol", right_on = "NSE symbol", how = "left")

In [122]:
# 23 companies are not captured by the Prowess data
mainDirect5.loc[ pd.isnull(mainDirect5["NSE symbol"]) ]["Symbol"].unique()

array(['IIFLCAPS', 'TRANSWORLD', 'GVT&D', 'INTERISE', 'ABREL',
       'FUTUREVENT', 'LMW', 'ALIVUS', 'ALLDIGI', 'AMBANIORG', 'VEEDOL',
       'TICL', 'ARVINDPORT', 'GMRAIRPORT', 'URAVIDEF', 'HBLENGINE',
       'TIPSMUSIC', 'ITTL', 'INDUSINVIT', 'NDRINVIT', 'SEITINVIT',
       'VINCOFE', 'SAME'], dtype=object)

In [123]:
mainDirect5["Skills/Competencies"] = mainDirect5["Skills/Competencies"].astype("str")
skills = pd.Series(list(set( a
                            for x in mainDirect5["Skills/Competencies"].dropna()
                            for a in x.split(",") )))
mainDirect5["Skills"] = [x.split(",") for x in mainDirect5["Skills/Competencies"]]
mainDirect5["Skills"]

0                                                     [nan]
1         [ACCOUNTS/AUDIT/TAXATION, CONSUMER DISCRETIONA...
2         [ACCOUNTS/AUDIT/TAXATION, CONSUMER DISCRETIONA...
3         [ACCOUNTS/AUDIT/TAXATION, CONSUMER DISCRETIONA...
4                                                     [nan]
                                ...                        
258252                                                [nan]
258253                                                [nan]
258254                                                [nan]
258255                                                [nan]
258256                                                [nan]
Name: Skills, Length: 258257, dtype: object

In [124]:
eduCols = [x for x in mainDirect5.columns if "Edu" in x]
mainDirect5["EducationAll"] = mainDirect5[eduCols].astype(str).agg(" ".join, axis=1)
mainDirect5["IsMBA"] = np.where(
    mainDirect5["EducationAll"].str.contains(('MBA|M.B.A|M. B. A|M B A'), case = False, na = False ) , 1,0)
mainDirect5["IsPhD"] = np.where(
    mainDirect5["EducationAll"].str.contains(('PhD|PH.D|P H D|P. H. D'), case = False, na = False ) , 1,0)

In [125]:
mainDirect5["EducationAll"]

0         GRADUATE: B.SC. nan nan nan nan nan nan nan na...
1         POST GRADUATE: M.B.A. POST GRADUATE: M.COM.,MA...
2         POST GRADUATE: M.B.A. POST GRADUATE: M.COM.,MA...
3         POST GRADUATE: M.B.A. POST GRADUATE: M.COM.,MA...
4         GRADUATE: ASSOCIATE MEMBER,INSTITUTE OF COMPAN...
                                ...                        
258252              nan nan nan nan nan nan nan nan nan nan
258253    GRADUATE: ASSOCIATE MEMBER,INSTITUTE OF COMPAN...
258254              nan nan nan nan nan nan nan nan nan nan
258255              nan nan nan nan nan nan nan nan nan nan
258256    GRADUATE: ASSOCIATE MEMBER,INSTITUTE OF CHARTE...
Name: EducationAll, Length: 258257, dtype: object

In [126]:
mainDirect5["IsMBA"].value_counts()

IsMBA
0    189925
1     68332
Name: count, dtype: int64

In [127]:
mainDirect5["IsPhD"].value_counts()

IsPhD
0    240447
1     17810
Name: count, dtype: int64

In [128]:
symbolNIC = mainDirect5[["Symbol", "NIC code"]].copy().drop_duplicates()
symbolNICdict = dict(zip(symbolNIC["Symbol"], symbolNIC["NIC code"]))

mainDirect5["AllPastDirectNIC"] = mainDirect5["AllPastDirect"].progress_apply(lambda list: set(symbolNICdict[x[0]] for x in list))
mainDirect5["CurrDirectANIC"] = mainDirect5["CurrDirectA"].progress_apply(lambda list: set(symbolNICdict[x[0]] for x in list))
mainDirect5["CurrDirectABNIC"] = mainDirect5["CurrDirectAB"].progress_apply(lambda list: set(symbolNICdict[x[0]] for x in list))

mainDirect5["AllNIC"] = mainDirect5.progress_apply(lambda row: 
                                                   (row["AllPastDirectNIC"] | row["CurrDirectANIC"] | row["CurrDirectABNIC"]), axis = 1)

100%|███████████████████████████████████████████████████████████████████████| 258257/258257 [00:06<00:00, 41255.05it/s]


In [129]:
# mainDirect5.loc[ mainDirect5["Symbol"] == "20MICRONS"]

In [130]:
mainDirect5["AllNIC_Industry"] = mainDirect5["AllNIC"].progress_apply( lambda s: set(x[:2] for x in s if isinstance(x, str)))

100%|██████████████████████████████████████████████████████████████████████| 258257/258257 [00:00<00:00, 535206.33it/s]


In [131]:
mainDirect5["HasFinanceXP"] = mainDirect5["AllNIC_Industry"].progress_apply( lambda s: 1
                                                                            if any(x in {"64", "65", "66"} for x in s)
                                                                            else 0)
mainDirect5["HasTechXP"]    = mainDirect5["AllNIC_Industry"].progress_apply( lambda s: 1
                                                                            if any(x in {"62", "63"} for x in s)
                                                                            else 0)

100%|██████████████████████████████████████████████████████████████████████| 258257/258257 [00:00<00:00, 684875.74it/s]


In [132]:
mainDirect5["HasRelatedIndustryXP"] = mainDirect5.progress_apply(lambda row: 
                                                                 1 if str(row["NIC code"])[:2] in row["AllPastDirectNIC"] else 0,
                                                                 axis = 1)

100%|███████████████████████████████████████████████████████████████████████| 258257/258257 [00:05<00:00, 45012.22it/s]


In [133]:
mainDirect5["IsExecCurrent"] = np.where((
    (mainDirect5["Position on Board"].str.contains("CHIEF", case = False, na = False ) &\
     mainDirect5["Position on Board"].str.contains("OFFICER", case = False, na = False )) |\
    (mainDirect5["Occupation"].str.contains("CHIEF", case = False, na = False ) &\
     mainDirect5["Occupation"].str.contains("OFFICER", case = False, na = False ))),
                                    1,0)

mainDirect5["NumExecAll"] = mainDirect5.groupby(["Person Code", "AsOnDate"])["IsExecCurrent"].transform("sum")

mainDirect5["IsOutsideExecXP"] = np.where( (mainDirect5["NumExecAll"] - mainDirect5["IsExecCurrent"]) > 0, 1, 0)

mainDirect5["HasExecXP"] = np.where( mainDirect5["NumExecAll"]>0, 1, 0)
mainDirect5["HasExecXP"] = mainDirect5.sort_values("AsOnDate").groupby(["Person Code"])["HasExecXP"].cumsum()
mainDirect5["HasExecXP"] = np.where( mainDirect5["HasExecXP"]>0, 1, 0)

mainDirect5["PublicExecXPDummy"] = np.where((mainDirect5["HasExecXP"] == 1)&(mainDirect5["Entity type"] == "Public Ltd."), 1, 0)
mainDirect5["PrivateExecXPDummy"] = np.where((mainDirect5["HasExecXP"] == 1)&(mainDirect5["Entity type"] == "Private Ltd."), 1, 0)

mainDirect5["PublicExecXPDummy"] = mainDirect5.groupby(["Person Code", "AsOnDate"])["PublicExecXPDummy"].transform("sum")
mainDirect5["PrivateExecXPDummy"] = mainDirect5.groupby(["Person Code", "AsOnDate"])["PrivateExecXPDummy"].transform("sum")

mainDirect5["PublicExecXPDummy"] = np.where( mainDirect5["PublicExecXPDummy"]>0, 1, 0)
mainDirect5["PrivateExecXPDummy"] = np.where( mainDirect5["PrivateExecXPDummy"]>0, 1, 0)

mainDirect5["HasPublicExecXP"] = mainDirect5.sort_values("AsOnDate").groupby(["Person Code"])["PublicExecXPDummy"].cumsum()
mainDirect5["HasPrivateExecXP"] = mainDirect5.sort_values("AsOnDate").groupby(["Person Code"])["PrivateExecXPDummy"].cumsum()

mainDirect5["HasPublicExecXP"] = np.where( mainDirect5["HasPublicExecXP"]>0, 1, 0)
mainDirect5["HasPrivateExecXP"] = np.where( mainDirect5["HasPrivateExecXP"]>0, 1, 0)

## 5 General Ability Index prereq 6

In [134]:
# past number of 
# (1) positions (current and past),
# (2) firms (past), and 
# (3) industries (past) in which a DIRECTOR worked; 
# (4) whether the DIRECTOR held a CEO/MD/Chairperson position at a different company; and 
# (5) whether the DIRECTOR worked for a conglomerate.

In [135]:
# (1) wrong
# mainDirect5["SymbolTerm"] = mainDirect5.progress_apply(lambda row: (row["Symbol"], row["TermNumber"]), axis = 1)
# allTermsHeld = mainDirect5.groupby(["Person Code", "AsOnDate"])["SymbolTerm"].progress_apply(tuple)
# allTermsHeld2 = allTermsHeld.to_frame().reset_index().rename({"SymbolTerm":"AllTermsHeld"}, axis = 1).drop_duplicates(
#     subset = ["Person Code", "AsOnDate", "AllTermsHeld"]).groupby(["Person Code", "AsOnDate"]).AllTermsHeld.progress_apply(
#     lambda x: list(accumulate(x, lambda a, b: a + b)))

# mainDirect6 = mainDirect5.merge(allTermsHeld2.to_frame().reset_index(), on = ["Person Code", "AsOnDate"])
# mainDirect6.loc[:, "TermsPastCurr"] = [x for sublist in mainDirect6["AllTermsHeld"] for x in sublist]

In [136]:
# mainDirect6.loc[:, "NumTermsPastCurr"] = mainDirect6["TermsPastCurr"].progress_apply(lambda x: len(x))

In [137]:
# (1)
skillList = [x.lower() for x in skills.dropna().tolist() if x != "nan"]
mainDirect5["Skills"] = mainDirect5["Skills"].progress_apply(lambda x: {skill.lower() for skill in x if skill != "nan"})

mainDirect5["SkillsInPositiononBoard"] = mainDirect5["Position on Board"].astype(str)\
.progress_apply( lambda x: {skill for skill in skillList if skill in x.lower()})

mainDirect5["SkillsInOccupation"] = mainDirect5["Occupation"].astype(str)\
.progress_apply( lambda x: {skill for skill in skillList if skill in x.lower()})

100%|██████████████████████████████████████████████████████████████████████| 258257/258257 [00:00<00:00, 261066.53it/s]


In [138]:
mainDirect5["AllSkills"] = mainDirect5.progress_apply(lambda row: set(row["Skills"] | row["SkillsInPositiononBoard"] | row["SkillsInOccupation"]), axis = 1)

100%|███████████████████████████████████████████████████████████████████████| 258257/258257 [00:06<00:00, 39707.66it/s]


In [139]:
finSkills = ["finance", "financial services", "accounts/audit/taxation"]
mainDirect5["HasTechSkill"] = mainDirect5["AllSkills"].progress_apply(lambda x: 1 if "information technology" in x else 0)
mainDirect5["HasFinanceSkill"] = mainDirect5["AllSkills"].progress_apply(lambda x: 1 if any(skill in finSkills for skill in x) else 0)

100%|██████████████████████████████████████████████████████████████████████| 258257/258257 [00:00<00:00, 569426.36it/s]


In [140]:
mainDirect5["HasFinanceSkill"] 

0         0
1         1
2         1
3         1
4         0
         ..
258252    0
258253    0
258254    0
258255    0
258256    0
Name: HasFinanceSkill, Length: 258257, dtype: int64

In [141]:
skillList

['diversified',
 'supply chain management',
 'diversity',
 'consumer discretionary',
 'legal/compliance',
 'energy',
 'leadership/management',
 'strategy',
 'accounts/audit/taxation',
 'fast moving consumer goods',
 'corporate social responsibility',
 'economics',
 'telecommunication',
 'operations',
 'public policy',
 'financial services',
 'research & development',
 'marketing',
 'corporate governance',
 'commodities',
 'human resources',
 'finance',
 'services',
 'personal values',
 'international business',
 'healthcare',
 'information technology',
 'utilities',
 'risk management',
 'industrials']

In [142]:
mainDirect5["FirmsPast"] = mainDirect5["AllPastDirect"].progress_apply( lambda s: set(x[0] for x in s) if s else pd.NA)

# (1)
mainDirect5["NumSkills"] = mainDirect5["AllSkills"].progress_apply(lambda x: len(x))

100%|█████████████████████████████████████████████████████████████████████| 258257/258257 [00:00<00:00, 1342699.89it/s]


In [143]:
# (2) & (3)
mainDirect5["NumFirmsPast"] = mainDirect5["FirmsPast"].progress_apply(lambda x: len(x) if isinstance(x, set) else 0)
mainDirect5["NumIndustryPast"] = mainDirect5["AllPastDirectNIC"].progress_apply(lambda x: len(x) if isinstance(x, set) else 0)

100%|█████████████████████████████████████████████████████████████████████| 258257/258257 [00:00<00:00, 1249654.90it/s]


In [144]:
# (4)
mainDirect5["HasCeoMDChairXP"] = np.where((mainDirect5["IsChairman"] == 1) & (mainDirect5["IsCeoMD"] == 1), 1, 0)
mainDirect5["HasCeoMDChairXP"] = mainDirect5.groupby(["Person Code", "AsOnDate"])["HasCeoMDChairXP"].transform("sum")
mainDirect5["HasCeoMDChairXP"] = np.where( mainDirect5["HasCeoMDChairXP"]>0, 1, 0)
mainDirect5["HasCeoMDChairXP"] = mainDirect5.sort_values("AsOnDate").groupby(["Person Code"])["HasCeoMDChairXP"].cumsum()
mainDirect5["HasCeoMDChairXP"] = np.where( mainDirect5["HasCeoMDChairXP"]>0, 1, 0)

In [145]:
# (5)
mainDirect5["HasConglomerateXP"] = mainDirect5["Ownership group"].progress_apply(lambda x: 1
                                                                                 if ((isinstance(x, str)) and ("Group" in x))
                                                                                 else 0 )
mainDirect5["HasConglomerateXP"] = mainDirect5.groupby(["Person Code", "AsOnDate"])["HasConglomerateXP"].transform("sum")
mainDirect5["HasConglomerateXP"] = np.where( mainDirect5["HasConglomerateXP"]>0, 1, 0)
mainDirect5["HasConglomerateXP"] = mainDirect5.sort_values("AsOnDate").groupby(["Person Code"])["HasConglomerateXP"].cumsum()
mainDirect5["HasConglomerateXP"] = np.where( mainDirect5["HasConglomerateXP"]>0, 1, 0)

100%|█████████████████████████████████████████████████████████████████████| 258257/258257 [00:00<00:00, 1195000.68it/s]


In [146]:
# mainDirect5.loc[ mainDirect5["Person Code"] == "P00033271" ]
# mainDirect5.loc[ mainDirect5["Person Code"] == "P00114429" ]
# mainDirect5.loc[ mainDirect5["Person Code"] == "P00075399" ]
# mainDirect5.loc[ (mainDirect5["Person Code"] == "P00004449")].sort_values("AsOnDate")
# mainDirect5.loc[ (mainDirect5["Person Code"] == "P00000249"), ["AsOnDate", "AllTermsHeld"]]
# mainDirect5.loc[ (mainDirect5["Person Code"] == "P00000249"), "AllTermsHeld_y"][1]

In [147]:
mainDirect5["HasCeoMDChairXP"].value_counts()

HasCeoMDChairXP
0    241136
1     17121
Name: count, dtype: int64

In [148]:
mainDirect5

,Symbol,Company,AsOnDate,AsOnYear,ISIN,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Member of Civil Services,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Occupation,Cessation Reason,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Brief Profile,Tenure Valid till,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,AppointDummy,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal,CloBalTotalXP,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMDPosition,IsChairmanPosition,IsCeoMDOccupation,IsChairmanOccupation,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManager,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsinCompIndep,TenureInYearsinCompTotal,IsFirstTerm,IsFirstTermIndep,IsZeroYear,IsZeroYearIndep,IsOneYear,IsOneYearIndep,IsTwoYear,IsTwoYearIndep,IsThreeYear,IsThreeYearIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,IsBusy,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,NSE symbol,NIC code,Entity type,Ownership group,Prowess company code,Company Name,Skills,EducationAll,IsMBA,IsPhD,AllPastDirectNIC,CurrDirectANIC,CurrDirectABNIC,AllNIC,AllNIC_Industry,HasFinanceXP,HasTechXP,HasRelatedIndustryXP,IsExecCurrent,NumExecAll,IsOutsideExecXP,HasExecXP,PublicExecXPDummy,PrivateExecXPDummy,HasPublicExecXP,HasPrivateExecXP,SkillsInPositiononBoard,SkillsInOccupation,AllSkills,HasTechSkill,HasFinanceSkill,FirmsPast,NumSkills,NumFirmsPast,NumIndustryPast,HasCeoMDChairXP,HasConglomerateXP
0,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027662,MR.,DIPTIMOY,DINESHCHANDRA,BHATTACHARJEE,1955-12-09,M,NaN,NaN,NaN,nan,nan,GRADUATE: B.SC.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,NOMINATION ENDED,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"MR.D.D.BHATTACHARJEE, IDBI-NOMINEE DIRECTOR HO...",NaT,U,2005-04-29,2009-03-02,NaT,2010-03-31,1,0.0,1.0,1.0,0,0.0,0.0,0.0,1403.0,0.0,0.0,1038.0,0.0,0.0,1038.0,1403.0,0.0,0.0,1403.0,1038.0,0.0,0.0,1038.0,1403.0,0.0,0.0,1403.0,1402,(),"{(20MICRONS, U)}","{(20MICRONS, U)}",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,69,0.00000,3.843836,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,20MICRONS,08,Public Ltd.,20 Microns Group,11.0,20 Microns Ltd.,{},GRADUATE: B.SC. nan nan nan nan nan nan nan na...,0,0,{},{08},{08},{08},{08},0,0,0,0,0,0,0,0,0,0,0,{},{},{},0,0,<NA>,0,0,0,0,1
1,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027633,MR.,RAMKISHAN,AMIRCHAND,DEVIDAYAL,1950-12-08,M,INDIA,NONE,NaN,nan,nan,POST GRADUATE: M.B.A.,"POST GRADUATE: M.COM.,MAHARAJA SAYAJIRAO UNIVE...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,CONSUMER DISC

# SKILLS

In [149]:
# if skills keywords map exists in allskills

mainDirect5["AllSkills"] = mainDirect5["AllSkills"].astype(str).str.lower()

skillset_keywords_map = { "sustainability": [
        "sustainability", "sustainable", "csr", "corporate social responsibility",
        "sustainability committee", "esg committee", "csr committee",
        "corporate social responsibility committee", "environment committee"
    ],
    "entrepreneurial": [
        "entrepreneur", "entrepreneurship", "startup", "founder", "co-founder",
        "innovation", "innovator", "venture", "investor"
    ],
    "compensation": [
        "compensation committee", "remuneration committee", "pay committee", "salary committee",
        "executive compensation", "incentive committee", "reward committee",
        "compensation and benefits", "compensation policy", "remuneration policy",
        "compensation", "salary", "remuneration", "pay", "bonus", "incentive",
        "performance bonus", "employee benefits", "wages", "rewards"
    ],
    "conglomerate_experience": [
        "conglomerate", "diversified group", "group company", "holding company",
        "multiple sectors", "business group", "corporate group", "inter-industry",
        "multi-industry", "parent-subsidiary structure", "group-wide strategy"
    ],
    "hr": [
        "human resources committee", "hr", "people management", "talent management",
        "organizational development", "performance management", "employee relations",
        "hr strategy", "compensation and benefits", "workforce planning",
        "succession planning", "human resource committee", "hr committee",
        "people committee", "talent committee", "organizational development committee",
        "compensation and benefits committee", "employee relations committee",
        "workforce planning committee", "succession planning committee",
        "diversities", "personal values"
    ],
    "technology": [
        "technology", "information technology", "it", "software engineering", "hardware", 
        "computer science", "tech", "digital transformation", "data science", "machine learning", 
        "ai", "artificial intelligence", "cybersecurity", "cloud computing", "systems architecture", 
        "technology strategy", "data analytics", "blockchain", "innovation", "info tech"
    ],
    "finance_accounting": [
        "finance", "financial", "accounting", "accounts", "chartered accountant", 
        "ca", "cfo", "chief financial officer", "auditor", "audit", "financial analysis", 
        "capital markets", "treasury", "banking", "investment", "equity", "debt", 
        "risk management", "taxation", "budgeting", "forecasting", "valuation", 
        "financial planning", "cost control", "internal controls", "compliance", 
        "financial reporting", "economics", "financial services", "commodities"
    ],
    "governance": [
        "governance", "corporate governance", "governance committee", "audit committee", 
        "risk committee", "ethics committee", "compliance committee", "board governance", 
        "regulatory affairs", "internal controls", "oversight", "governance and compliance", 
        "governance practices", "governance framework", "governance structure", 
        "governance responsibilities"
    ],
    "government_policy": [
        "government", "policy", "government affairs", "public policy", "government relations", 
        "legislative affairs", "policy making", "public sector", "regulations", 
        "government strategy", "public administration", "government services", 
        "policy development", "political affairs", "utilities"
    ],
    "international": [
        "international", "global", "international relations", "cross-border", "foreign affairs", 
        "multinational", "overseas", "global strategy", "global expansion", 
        "international business", "cross-cultural", "international markets", 
        "global trade", "global governance"
    ],
    "leadership": [
        "leadership", "executive leadership", "team leadership", "leadership development", 
        "strategic leadership", "leadership skills", "leadership team", 
        "senior leadership", "leadership strategy", "leadership responsibilities", 
        "visionary leadership", "leadership roles", "leadership training", 
        "leadership capabilities", "leadership and management", "senior management"
    ],
    "legal": [
        "legal", "legal affairs", "legal compliance", "legal counsel", "corporate legal", 
        "legal strategy", "legal advisor", "contract law", "intellectual property", 
        "regulatory compliance", "litigation", "legal framework", "lawsuit", 
        "legal governance", "legal risk", "compliance"
    ],
    "marketing": [
        "marketing", "marketing strategy", "digital marketing", "marketing campaign", 
        "marketing communications", "brand management", "customer engagement", 
        "marketing research", "social media marketing", "advertising", 
        "marketing manager", "market research", "content marketing", 
        "product marketing", "marketing analysis", "markets", 
        "fast moving consumer goods", "fmcg", "services", "consumer differentiation"
    ],
    "risk_management": [
        "risk management", "risk mitigation", "risk assessment", "risk committee", 
        "operational risk", "financial risk", "strategic risk", "compliance risk", 
        "risk control", "enterprise risk management", "risk governance", 
        "risk monitoring", "risk management strategy", "risk management framework", 
        "risk prevention"
    ],
    "scientific": [
        "scientific", "research and development", "r&d", "innovation", 
        "scientific research", "scientific committee", "technology research", 
        "science strategy", "scientific discovery", "scientific innovation", 
        "scientific analysis", "biotech", "scientific approach", "lab research", 
        "clinical research"
    ],
    "strategic_planning": [
        "strategic planning", "strategic development", "strategic initiatives", 
        "business strategy", "strategic goals", "long-term planning", 
        "corporate strategy", "strategic management", "strategic vision", 
        "strategic objectives", "strategy formulation", "strategic direction", 
        "strategic partnerships", "business planning", "strategic forecasting", 
        "strategy"
    ],
    "manufacturing_supply_chain": [
        "manufacturing", "production", "operations", "supply chain", "factory",
        "industrial", "plant", "manufacturing unit", "process engineering",
        "assembly line", "lean manufacturing", "six sigma", "quality control",
        "industrial engineering", "materials management", "shop floor",
        "production planning", "logistics", "automation", "mechanical engineering",
        "process optimization", "machinery", "manufacturing strategy",
        "supply chain management", "engineering operations",
        "operational management", "project management"
    ]
}

for col, keywords in skillset_keywords_map.items():
    skilllist_col = f"skilllist_{col}"
    mainDirect5[skilllist_col] = mainDirect5["AllSkills"].apply(
        lambda x: int(any(kw in x for kw in keywords)))
    mainDirect5.groupby(["Person Code", "AsOnDate"])[skilllist_col].transform('any')
    mainDirect5.groupby("Person Code")[skilllist_col].ffill()


In [150]:
mainDirect5

,Symbol,Company,AsOnDate,AsOnYear,ISIN,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Member of Civil Services,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Occupation,Cessation Reason,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Brief Profile,Tenure Valid till,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,AppointDummy,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal,CloBalTotalXP,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMDPosition,IsChairmanPosition,IsCeoMDOccupation,IsChairmanOccupation,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManager,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsinCompIndep,TenureInYearsinCompTotal,IsFirstTerm,IsFirstTermIndep,IsZeroYear,IsZeroYearIndep,IsOneYear,IsOneYearIndep,IsTwoYear,IsTwoYearIndep,IsThreeYear,IsThreeYearIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,IsBusy,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,NSE symbol,NIC code,Entity type,Ownership group,Prowess company code,Company Name,Skills,EducationAll,IsMBA,IsPhD,AllPastDirectNIC,CurrDirectANIC,CurrDirectABNIC,AllNIC,AllNIC_Industry,HasFinanceXP,HasTechXP,HasRelatedIndustryXP,IsExecCurrent,NumExecAll,IsOutsideExecXP,HasExecXP,PublicExecXPDummy,PrivateExecXPDummy,HasPublicExecXP,HasPrivateExecXP,SkillsInPositiononBoard,SkillsInOccupation,AllSkills,HasTechSkill,HasFinanceSkill,FirmsPast,NumSkills,NumFirmsPast,NumIndustryPast,HasCeoMDChairXP,HasConglomerateXP,skilllist_sustainability,skilllist_entrepreneurial,skilllist_compensation,skilllist_conglomerate_experience,skilllist_hr,skilllist_technology,skilllist_finance_accounting,skilllist_governance,skilllist_government_policy,skilllist_international,skilllist_leadership,skilllist_legal,skilllist_marketing,skilllist_risk_management,skilllist_scientific,skilllist_strategic_planning,skilllist_manufacturing_supply_chain
0,20MICRONS,20 MICRONS LTD.,2009-03-31,2009,INE144J01027,P00027662,MR.,DIPTIMOY,DINESHCHANDRA,BHATTACHARJEE,1955-12-09,M,NaN,NaN,NaN,nan,nan,GRADUATE: B.SC.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,NOMINATION ENDED,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"MR.D.D.BHATTACHARJEE, IDBI-NOMINEE DIRECTOR HO...",NaT,U,2005-04-29,2009-03-02,NaT,2010-03-31,1,0.0,1.0,1.0,0,0.0,0.0,0.0,1403.0,0.0,0.0,1038.0,0.0,0.0,1038.0,1403.0,0.0,0.0,1403.0,1038.0,0.0,0.0,1038.0,1403.0,0.0,0.0,1403.0,1402,(),"{(20MICRONS, U)}","{(20MICRONS, U)}",(),(),0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,N,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,69,0.00000,3.843836,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,20MICRONS,08,Public Ltd.,20 Microns Group,11.0,20 Microns Lt

In [151]:
mainDirect5['Academic'] = mainDirect5['IsPhD']

In [152]:
def extract_manufacturing(nic_code):
    if pd.isnull(nic_code) or str(nic_code).strip() == '':
        return 0
    try:
        nic_str = str(int(float(nic_code))).zfill(2) 
        nic2 = int(nic_str[:2]) 
        return 1 if 10 <= nic2 <= 33 else 0 
    except:
        return 0
mainDirect5['Manufacturing'] = mainDirect5['NIC code'].progress_apply(extract_manufacturing)

100%|██████████████████████████████████████████████████████████████████████| 258257/258257 [00:00<00:00, 368766.72it/s]


In [153]:
mainDirect5['Outside Board'] = mainDirect5['CompCountOtherPastTotalAB'].apply(lambda x: 1 if x > 0 else 0)

In [154]:
mainDirect5["Company Business"] = mainDirect5["HasRelatedIndustryXP"]

## Committees and Brief Profile skillset parse

In [155]:
mainComm = pd.read_pickle(rf"{input_folder_path}\Main_Committees_incl Cess.pkl")

In [156]:
mainComm["Committee_position"] = list(zip(mainComm["Committee Name"], mainComm["Position"]))

In [157]:
all_committee_position = mainComm.groupby(
    ["Person Code", "Symbol", "AsOnDate"])["Committee_position"].agg(list).reset_index()

all_committee = mainComm.groupby(
    ["Person Code", "Symbol", "AsOnDate"])["Committee Name"].agg(list).reset_index()

In [158]:
keyword_categories = { "sustainability": [
        "sustainability", "sustainable", "csr", "corporate social responsibility",
        "sustainability committee", "esg committee", "csr committee",
        "corporate social responsibility committee", "environment committee"
    ],
    "entrepreneurial": [
        "entrepreneur", "entrepreneurship", "startup", "founder", "co-founder",
        "innovation", "innovator", "venture", "investor"
    ],
    "compensation": [
        "compensation committee", "remuneration committee", "pay committee", "salary committee",
        "executive compensation", "incentive committee", "reward committee",
        "compensation and benefits", "compensation policy", "remuneration policy",
        "compensation", "salary", "remuneration", "pay", "bonus", "incentive",
        "performance bonus", "employee benefits", "wages", "rewards"
    ],
    "conglomerate_experience": [
        "conglomerate", "diversified group", "group company", "holding company",
        "multiple sectors", "business group", "corporate group", "inter-industry",
        "multi-industry", "parent-subsidiary structure", "group-wide strategy"
    ],
    "hr": [
        "human resources committee", "hr", "people management", "talent management",
        "organizational development", "performance management", "employee relations",
        "hr strategy", "compensation and benefits", "workforce planning",
        "succession planning", "human resource committee", "hr committee",
        "people committee", "talent committee", "organizational development committee",
        "compensation and benefits committee", "employee relations committee",
        "workforce planning committee", "succession planning committee",
        "diversities", "personal values"
    ],
    "technology": [
        "technology", "information technology", "it", "software engineering", "hardware", 
        "computer science", "tech", "digital transformation", "data science", "machine learning", 
        "ai", "artificial intelligence", "cybersecurity", "cloud computing", "systems architecture", 
        "technology strategy", "data analytics", "blockchain", "innovation", "info tech"
    ],
    "finance_accounting": [
        "finance", "financial", "accounting", "accounts", "chartered accountant", 
        "ca", "cfo", "chief financial officer", "auditor", "audit", "financial analysis", 
        "capital markets", "treasury", "banking", "investment", "equity", "debt", 
        "risk management", "taxation", "budgeting", "forecasting", "valuation", 
        "financial planning", "cost control", "internal controls", "compliance", 
        "financial reporting", "economics", "financial services", "commodities"
    ],
    "governance": [
        "governance", "corporate governance", "governance committee", "audit committee", 
        "risk committee", "ethics committee", "compliance committee", "board governance", 
        "regulatory affairs", "internal controls", "oversight", "governance and compliance", 
        "governance practices", "governance framework", "governance structure", 
        "governance responsibilities"
    ],
    "government_policy": [
        "government", "policy", "government affairs", "public policy", "government relations", 
        "legislative affairs", "policy making", "public sector", "regulations", 
        "government strategy", "public administration", "government services", 
        "policy development", "political affairs", "utilities"
    ],
    "international": [
        "international", "global", "international relations", "cross-border", "foreign affairs", 
        "multinational", "overseas", "global strategy", "global expansion", 
        "international business", "cross-cultural", "international markets", 
        "global trade", "global governance"
    ],
    "leadership": [
        "leadership", "executive leadership", "team leadership", "leadership development", 
        "strategic leadership", "leadership skills", "leadership team", 
        "senior leadership", "leadership strategy", "leadership responsibilities", 
        "visionary leadership", "leadership roles", "leadership training", 
        "leadership capabilities", "leadership and management", "senior management"
    ],
    "legal": [
        "legal", "legal affairs", "legal compliance", "legal counsel", "corporate legal", 
        "legal strategy", "legal advisor", "contract law", "intellectual property", 
        "regulatory compliance", "litigation", "legal framework", "lawsuit", 
        "legal governance", "legal risk", "compliance"
    ],
    "marketing": [
        "marketing", "marketing strategy", "digital marketing", "marketing campaign", 
        "marketing communications", "brand management", "customer engagement", 
        "marketing research", "social media marketing", "advertising", 
        "marketing manager", "market research", "content marketing", 
        "product marketing", "marketing analysis", "markets", 
        "fast moving consumer goods", "fmcg", "services", "consumer differentiation"
    ],
    "risk_management": [
        "risk management", "risk mitigation", "risk assessment", "risk committee", 
        "operational risk", "financial risk", "strategic risk", "compliance risk", 
        "risk control", "enterprise risk management", "risk governance", 
        "risk monitoring", "risk management strategy", "risk management framework", 
        "risk prevention"
    ],
    "scientific": [
        "scientific", "research and development", "r&d", "innovation", 
        "scientific research", "scientific committee", "technology research", 
        "science strategy", "scientific discovery", "scientific innovation", 
        "scientific analysis", "biotech", "scientific approach", "lab research", 
        "clinical research"
    ],
    "strategic_planning": [
        "strategic planning", "strategic development", "strategic initiatives", 
        "business strategy", "strategic goals", "long-term planning", 
        "corporate strategy", "strategic management", "strategic vision", 
        "strategic objectives", "strategy formulation", "strategic direction", 
        "strategic partnerships", "business planning", "strategic forecasting", 
        "strategy"
    ],
    "manufacturing_supply_chain": [
        "manufacturing", "production", "operations", "supply chain", "factory",
        "industrial", "plant", "manufacturing unit", "process engineering",
        "assembly line", "lean manufacturing", "six sigma", "quality control",
        "industrial engineering", "materials management", "shop floor",
        "production planning", "logistics", "automation", "mechanical engineering",
        "process optimization", "machinery", "manufacturing strategy",
        "supply chain management", "engineering operations",
        "operational management", "project management"
    ]
}

keyword_categories = {
    category.lower(): [kw.lower() for kw in keywords]
    for category, keywords in keyword_categories.items()
}

for category, keywords in keyword_categories.items():
    col_name = f"committee_{category}"
    all_committee[col_name] = all_committee['Committee Name'].apply(
        lambda names: int(any(
            keyword in name.lower().strip()
            for name in names
            for keyword in keywords
        )) if isinstance(names,list) else 0
    )


for category, keywords in keyword_categories.items():
    col_name = f"profile_{category}"
    mainDirect5[col_name] = mainDirect5['Brief Profile'].astype(str).str.lower().apply(
        lambda x: int(any(keyword in x for keyword in keywords))
    )

mainDirect6 = mainDirect5.merge(
    all_committee,
    on=['Symbol', 'Person Code', 'AsOnDate'],
    how='left'
)

for category in keyword_categories.keys():
    committee_col = f"committee_{category}"
    profile_col = f"profile_{category}"
    mainDirect6[committee_col] = mainDirect6[committee_col].fillna(0).astype(int)
    mainDirect6[profile_col] = mainDirect6[profile_col].fillna(0).astype(int)
    
    mainDirect6.groupby(["Person Code", "AsOnDate"])[committee_col].transform('any')
    mainDirect6.groupby(["Person Code", "AsOnDate"])[profile_col].transform('any')
    
    mainDirect6.groupby("Person Code")[committee_col].ffill()
    mainDirect6.groupby("Person Code")[profile_col].ffill()
    


for category in keyword_categories.keys():
    skilllist_col = f"skilllist_{category}"
    committee_col = f"committee_{category}"
    profile_col = f"profile_{category}"
    
    combined_col = f"combined_{category}"
    
    mainDirect6[combined_col] = mainDirect6[[skilllist_col, committee_col, profile_col]].max(axis=1)

In [159]:
combined_skillsets_list = ["Academic",
 "Company Business",
 "combined_sustainability",
 "combined_entrepreneurial",
 "combined_compensation",
 "combined_conglomerate_experience",
 "combined_hr",
 "combined_technology",
 "combined_finance_accounting",
 "combined_governance",
 "combined_government_policy",
 "combined_international",
 "combined_leadership",
 "combined_legal",
 "combined_marketing",
 "combined_risk_management",
 "combined_scientific",
 "combined_strategic_planning",
 "combined_manufacturing_supply_chain"]

mainDirect6["combined_manufacturing_supply_chain"] = (mainDirect6["combined_manufacturing_supply_chain"] | mainDirect6["Manufacturing"]).astype(int)
mainDirect6["NumSkills_dummies"] = mainDirect6[combined_skillsets_list].sum(axis =1)

In [160]:
mainDirect6.columns.to_list()

['Symbol',
 'Company',
 'AsOnDate',
 'AsOnYear',
 'ISIN',
 'Person Code',
 'Director Salutation',
 'Director First Name',
 'Director Middle Name',
 'Director Surname',
 'Date of Birth',
 'Gender',
 'Nationality',
 'Member of Civil Services',
 'Promoter Director (Yes/No)',
 'Position on Board',
 'Independent (Yes/No)',
 'Education1',
 'Education2',
 'Education3',
 'Education4',
 'Education5',
 'Education6',
 'Education7',
 'Education8',
 'Education9',
 'Education10',
 'Skills/Competencies',
 'Occupation',
 'Cessation Reason',
 'Other Directorship 1',
 'Other Directorship 2',
 'Other Directorship 3',
 'Other Directorship 4',
 'Other Directorship 5',
 'Other Directorship 6',
 'Other Directorship 7',
 'Other Directorship 8',
 'Other Directorship 9',
 'Other Directorship 10',
 'Other Directorship 11',
 'Other Directorship 12',
 'Other Directorship 13',
 'Other Directorship 14',
 'Other Directorship 15',
 'Brief Profile',
 'Tenure Valid till',
 'Indep',
 'Appointment Date',
 'Cessation Date'

# Board Attendance

In [161]:
mainBoard = pd.read_pickle(rf"{input_folder_path}\Main_Board Attendance_incl Cess.pkl")

In [162]:
import re
mainBoard = mainBoard.dropna(subset = ["No.of Meetings Attended", "Total No.of Board Meetings Held"]).reset_index(drop = True)
mainBoard["No.of Meetings Attended"] = [float(re.sub(r"[a-zA-Z]", "", str(x)))
                                        for x in mainBoard["No.of Meetings Attended"]]

mainBoard

,Symbol,Company,AsOnDate,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Financial Year,Total No.of Board Meetings Held,No. of Meeting held during the tenure of the Director,No.of Meetings Attended
0,20MICRONS,20 MICRONS LTD.,2015-03-31,P00027633,MR.,RAMKISHAN,AMIRCHAND,DEVIDAYAL,2014-15,6.0,6,6.0
1,20MICRONS,20 MICRONS LTD.,2015-03-31,P00064917,MS.,DARSHA,RAJESH,KIKANI,2014-15,6.0,4,3.0
2,20MICRONS,20 MICRONS LTD.,2015-03-31,P00042601,MR.,NARESH,S.,MAKHIJA,2014-15,6.0,4,1.0
3,20MICRONS,20 MICRONS LTD.,2015-03-31,P00029006,MR.,ATIL,CHANDRESH,PARIKH,2014-15,6.0,6,4.0
4,20MICRONS,20 MICRONS LTD.,2015-03-31,P00027661,LATE MR.,CHANDRESH,SHANTILAL,PARIKH,2014-15,6.0,6,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
145237,ZYLOG,ZYLOG SYSTEMS LTD.,2016-03-31,P00036956,MR.,KUNJURI SATYA NARAYANA MURTY,SARVOTHAMA,RAO,2015-16,9.0,3,2.0
145238,ZYLOG,ZYLOG SYSTEMS LTD.,2016-03-31,P00027524,MR.,RAMANUJAM,NaN,SESHARATHNAM,2015-16,9.0,8,3.0
145239,ZYLOG,ZYLOG SYSTEMS LTD.,2016-03-31,P00075399,MR.,S.,P.,SRIHARI,2015-16,9.0,3,3.0
145240,ZYLOG,ZYLOG SYSTEMS LTD.,2016-03-31,P00062851,MR.,K.,S.,SUBRAMANIAN,2015-16,9.0,8,4.0


In [163]:
mainBoard["percent_board_absence"] = (mainBoard["Total No.of Board Meetings Held"] - mainBoard["No.of Meetings Attended"]) / mainBoard["Total No.of Board Meetings Held"]
mainBoard["percent_board_absence"] = mainBoard["percent_board_absence"].fillna(0)

In [164]:
mainBoard2 = mainBoard.drop(["Director Salutation", "Director First Name", "Director Middle Name", "Director Surname",
                             "Financial Year", "No. of Meeting held during the tenure of the Director", "Company"], axis = 1).drop_duplicates()


In [165]:
mainBoard2

,Symbol,AsOnDate,Person Code,Total No.of Board Meetings Held,No.of Meetings Attended,percent_board_absence
0,20MICRONS,2015-03-31,P00027633,6.0,6.0,0.000000
1,20MICRONS,2015-03-31,P00064917,6.0,3.0,0.500000
2,20MICRONS,2015-03-31,P00042601,6.0,1.0,0.833333
3,20MICRONS,2015-03-31,P00029006,6.0,4.0,0.333333
4,20MICRONS,2015-03-31,P00027661,6.0,6.0,0.000000
...,...,...,...,...,...,...
145237,ZYLOG,2016-03-31,P00036956,9.0,2.0,0.777778
145238,ZYLOG,2016-03-31,P00027524,9.0,3.0,0.666667
145239,ZYLOG,2016-03-31,P00075399,9.0,3.0,0.666667
145240,ZYLOG,2016-03-31,P00062851,9.0,4.0,0.555556


In [166]:
mainBoard2.loc[ mainBoard2["Person Code"] == "P00027508"]

,Symbol,AsOnDate,Person Code,Total No.of Board Meetings Held,No.of Meetings Attended,percent_board_absence
145231,ZYLOG,2015-03-31,P00027508,6.0,1.0,0.833333
145235,ZYLOG,2016-03-31,P00027508,9.0,1.0,0.888889


In [167]:
#handling duplicated rows erfjhbdfijbdsfjbsdkfvnb

totalBoardMeetingsmaxidx = mainBoard2.groupby(["Symbol", "Person Code", "AsOnDate"])["Total No.of Board Meetings Held"].idxmax()
mainBoard3 = mainBoard2.loc[totalBoardMeetingsmaxidx]

In [168]:
mainBoard3

,Symbol,AsOnDate,Person Code,Total No.of Board Meetings Held,No.of Meetings Attended,percent_board_absence
83,20MICRONS,2024-03-31,P00004449,5.0,5.0,0.000000
0,20MICRONS,2015-03-31,P00027633,6.0,6.0,0.000000
10,20MICRONS,2016-03-31,P00027633,10.0,8.0,0.200000
19,20MICRONS,2017-03-31,P00027633,5.0,5.0,0.000000
28,20MICRONS,2018-03-31,P00027633,4.0,4.0,0.000000
...,...,...,...,...,...,...
145233,ZYLOG,2015-03-31,P00062851,6.0,6.0,0.000000
145240,ZYLOG,2016-03-31,P00062851,9.0,4.0,0.555556
145241,ZYLOG,2016-03-31,P00075398,9.0,3.0,0.666667
145239,ZYLOG,2016-03-31,P00075399,9.0,3.0,0.666667


In [169]:
mainDirect7 = mainDirect6.merge(
    mainBoard3,
    on=["Symbol", "Person Code", "AsOnDate"],
    how="left"
)

In [170]:
# mainDirect7[ (mainDirect7["Symbol"] == "ATLASCYCLE")&(mainDirect7["AsOnDate"] == "2021-03-31")]

# Final Data Validation

In [171]:
mainDirect7.replace([np.inf, -np.inf], np.nan, inplace=True)

# EXPORTING

In [172]:
mainDirect7.to_pickle(rf"{output_folder_path}\Main_Director_COMPLETE.pkl")

In [173]:
listCol = [
    "AllPastDirect", "CurrDirectA", "CurrDirectAB",
    "PastSiezedDirectA", "PastSiezedDirectAB",
    "Skills",
    "AllPastDirectNIC",
    "CurrDirectANIC","CurrDirectABNIC",
    "AllNIC", "AllNIC_Industry",
    "SkillsInPositiononBoard", "SkillsInOccupation", "AllSkills",
    "FirmsPast"
]

mainDirect8 = mainDirect7.drop(listCol, axis = 1)


mainDirect8

,Symbol,Company,AsOnDate,AsOnYear,ISIN,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Member of Civil Services,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Occupation,Cessation Reason,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Brief Profile,Tenure Valid till,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,AppointDummy,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal,CloBalTotalXP,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMDPosition,IsChairmanPosition,IsCeoMDOccupation,IsChairmanOccupation,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManager,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsinCompIndep,TenureInYearsinCompTotal,IsFirstTerm,IsFirstTermIndep,IsZeroYear,IsZeroYearIndep,IsOneYear,IsOneYearIndep,IsTwoYear,IsTwoYearIndep,IsThreeYear,IsThreeYearIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,IsBusy,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,NSE symbol,NIC code,Entity type,Ownership group,Prowess company code,Company Name,EducationAll,IsMBA,IsPhD,HasFinanceXP,HasTechXP,HasRelatedIndustryXP,IsExecCurrent,NumExecAll,IsOutsideExecXP,HasExecXP,PublicExecXPDummy,PrivateExecXPDummy,HasPublicExecXP,HasPrivateExecXP,HasTechSkill,HasFinanceSkill,NumSkills,NumFirmsPast,NumIndustryPast,HasCeoMDChairXP,HasConglomerateXP,skilllist_sustainability,skilllist_entrepreneurial,skilllist_compensation,skilllist_conglomerate_experience,skilllist_hr,skilllist_technology,skilllist_finance_accounting,skilllist_governance,skilllist_government_policy,skilllist_international,skilllist_leadership,skilllist_legal,skilllist_marketing,skilllist_risk_management,skilllist_scientific,skilllist_strategic_planning,skilllist_manufacturing_supply_chain,Academic,Manufacturing,Outside Board,Company Business,profile_sustainability,profile_entrepreneurial,profile_compensation,profile_conglomerate_experience,profile_hr,profile_technology,profile_finance_accounting,profile_governance,profile_government_policy,profile_international,profile_leadership,profile_legal,profile_marketing,profile_risk_management,profile_scientific,profile_strategic_planning,profile_manufacturing_supply_chain,Committee Name,committee_sustainability,committee_entrepreneurial,committee_compensation,committee_conglomerate_experience,committee_hr,committee_technology,committee_finance_accounting,committee_governance,committee_government_policy,committee_international,committee_leadership,committee_legal,committee_marketing,committee_risk_management,committee_scientific,committee_strategic_planning,committee_manufacturing_supply_chain,combined_sustainability,combined_entrepreneurial,combined_compensation,combined_conglomerate_experience,

In [174]:
mainDirect8.to_csv(rf"{output_folder_path}\Main_Director_COMPLETE v190525.csv")

--------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------

# DATA ANALYSIS ----> PCA processing

In [175]:
pcaData = mainDirect7[[
    "Person Code", "Symbol", "AsOnDate",
    "Academic","combined_manufacturing_supply_chain","Outside Board",
    "combined_sustainability", "combined_entrepreneurial", "combined_compensation",
    "combined_governance", "combined_government_policy", "combined_international",
    "combined_leadership", "combined_legal",
    "combined_marketing", "combined_risk_management", "combined_scientific",
    "combined_strategic_planning", "combined_conglomerate_experience",
    "combined_hr", "combined_technology", "combined_finance_accounting","Company Business" ]].copy().set_index(["Person Code", "Symbol", "AsOnDate"], drop = True)

In [176]:
scaled_data = preprocessing.scale(pcaData)
pca = PCA(n_components = 4)
pca.fit(scaled_data)
pca_data = pca.transform(scaled_data)

pca_data_scaled = preprocessing.scale(pca_data)


eigenvalues = pca.explained_variance_
eigenvectors = pca.components_

In [177]:
pcaOutputNonScaled = pd.DataFrame(data = pca_data, index = pcaData.index)\
.rename({0:"General Ability Index"}, axis = 1).reset_index(drop = False)

pcaOutputScaled = pd.DataFrame(data = pca_data_scaled, index = pcaData.index)\
.rename({0:"General Ability Index_scaled"}, axis = 1).reset_index(drop = False)

pcaOutput = pcaOutputNonScaled.merge(pcaOutputScaled, on = ["Person Code", "Symbol", "AsOnDate"], how = "inner")
pcaOutput

,Person Code,Symbol,AsOnDate,General Ability Index,1_x,2_x,3_x,General Ability Index_scaled,1_y,2_y,3_y
0,P00027662,20MICRONS,2009-03-31,-1.981142,0.074610,-0.003697,-1.120158,-0.850692,0.065607,-0.003409,-1.055558
1,P00027633,20MICRONS,2009-03-31,2.111865,-1.444338,0.542887,-0.980042,0.906824,-1.270047,0.500483,-0.923522
2,P00027633,BANCOINDIA,2009-03-31,2.111865,-1.444338,0.542887,-0.980042,0.906824,-1.270047,0.500483,-0.923522
3,P00027633,MUNJALAU,2009-03-31,2.111865,-1.444338,0.542887,-0.980042,0.906824,-1.270047,0.500483,-0.923522
4,P00027664,20MICRONS,2009-03-31,-0.845518,1.084734,-0.647744,-0.247376,-0.363061,0.953837,-0.597149,-0.233109
...,...,...,...,...,...,...,...,...,...,...,...
258252,P00081419,ZYLOG,2018-03-31,-4.462567,-1.350199,-0.172527,2.358387,-1.916203,-1.187268,-0.159051,2.222377
258253,P00114429,ZYLOG,2018-03-31,-4.462567,-1.350199,-0.172527,2.358387,-1.916203,-1.187268,-0.159051,2.222377
258254,P00080494,ZYLOG,2018-03-31,-4.462567,-1.350199,-0.172527,2.358387,-1.916203,-1.187268,-0.159051,2.222377
258255,P00067111,ZYLOG,2018-03-31,-4.462567,-1.350199,-0.172527,2.358387,-1.916203,-1.187268,-0.159051,2.222377


In [178]:
pcaOutput.describe()

,AsOnDate,General Ability Index,1_x,2_x,3_x,General Ability Index_scaled,1_y,2_y,3_y
count,258257,2.582570e+05,2.582570e+05,2.582570e+05,2.582570e+05,2.582570e+05,2.582570e+05,2.582570e+05,2.582570e+05
mean,2016-06-24 12:40:52.422199552,-8.804163e-18,5.722706e-18,1.837869e-16,-1.364645e-17,7.031005e-15,6.902024e-15,2.870377e-15,-8.621477e-16
min,2006-03-31 00:00:00,-4.462567e+00,-2.983271e+00,-3.089321e+00,-1.718709e+00,-1.916203e+00,-2.623274e+00,-2.848019e+00,-1.619590e+00
25%,2012-03-31 00:00:00,-1.973517e+00,-8.662598e-01,-5.116856e-01,-8.487927e-01,-8.474180e-01,-7.617266e-01,-4.717186e-01,-7.998424e-01
50%,2017-03-31 00:00:00,2.332511e-01,-5.806842e-02,-3.110487e-02,-1.421786e-01,1.001568e-01,-5.106120e-02,-2.867532e-02,-1.339790e-01
75%,2021-03-31 00:00:00,2.053435e+00,6.671875e-01,3.499616e-01,6.300628e-01,8.817343e-01,5.866767e-01,3.226266e-01,5.937267e-01
max,2024-03-31 00:00:00,4.750028e+00,5.604914e+00,6.909137e+00,3.620842e+00,2.039637e+00,4.928558e+00,6.369475e+00,3.412026e+00
std,NaN,2.328864e+00,1.137234e+00,1.084728e+00,1.061202e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00


In [179]:
explained_variance = pca.explained_variance_ratio_
explained_variance

array([0.27117927, 0.06465913, 0.05882172, 0.05630533])

In [180]:
eigenvalues

array([5.42360636, 1.29318759, 1.17643903, 1.12611095])

In [181]:
eigenvectors

array([[ 0.03818636,  0.13379918,  0.12204446,  0.23101495,  0.14968743,
         0.13051981,  0.33725735,  0.20863192,  0.21377447,  0.3416969 ,
         0.31497378,  0.26916588,  0.33024262,  0.05651986,  0.33613559,
         0.0445428 ,  0.20864667,  0.21765681,  0.25181569,  0.00229927],
       [ 0.23598874, -0.12898113, -0.02001389,  0.11434334,  0.48789449,
         0.42267957,  0.00290853, -0.13207758, -0.01884906, -0.14548489,
        -0.21747809, -0.10211529, -0.16476777,  0.36087113, -0.18893465,
         0.0127566 , -0.08322107,  0.34797403,  0.28164192,  0.01521757],
       [ 0.48733097,  0.18351927, -0.06933947, -0.13761356, -0.07124433,
        -0.46590858, -0.19014372,  0.12268398,  0.27910754, -0.00420736,
        -0.02465292,  0.07838469, -0.05198771,  0.54317531,  0.01542054,
         0.14738759,  0.1058088 , -0.01178777, -0.03565407, -0.12145721],
       [ 0.1377704 , -0.04473563,  0.19561763,  0.28437598,  0.26611789,
         0.25112128,  0.07757732,  0.05106963,  

In [182]:
pcaOutput["General Ability Index"].var()

5.423606399828709

In [183]:
pcaOutput["FactorLoading1"] = eigenvectors[0, 0]
pcaOutput["FactorLoading2"] = eigenvectors[0, 1]
pcaOutput["FactorLoading3"] = eigenvectors[0, 2]
pcaOutput["FactorLoading4"] = eigenvectors[0, 3]
pcaOutput["FactorLoading5"] = eigenvectors[0, 4]

pcaOutput["FactorLoading6"] = eigenvectors[0, 5]
pcaOutput["FactorLoading7"] = eigenvectors[0, 6]
pcaOutput["FactorLoading8"] = eigenvectors[0, 7]
pcaOutput["FactorLoading9"] = eigenvectors[0, 8]
pcaOutput["FactorLoading10"] = eigenvectors[0, 9]

pcaOutput["FactorLoading11"] = eigenvectors[0, 10]
pcaOutput["FactorLoading12"] = eigenvectors[0, 11]
pcaOutput["FactorLoading13"] = eigenvectors[0, 12]
pcaOutput["FactorLoading14"] = eigenvectors[0, 13]
pcaOutput["FactorLoading15"] = eigenvectors[0, 14]

pcaOutput["FactorLoading16"] = eigenvectors[0, 15]
pcaOutput["FactorLoading17"] = eigenvectors[0, 16]
pcaOutput["FactorLoading18"] = eigenvectors[0, 17]
pcaOutput["FactorLoading19"] = eigenvectors[0, 18]
pcaOutput["FactorLoading20"] = eigenvectors[0, 19]

pcaOutput["Eigenvalue"] = eigenvalues[0]

In [184]:
scaled_data

array([[-0.27215874,  0.58515871, -0.81275526, ...,  0.31605803,
         0.38412079, -0.2287056 ],
       [-0.27215874,  0.58515871, -0.81275526, ...,  0.31605803,
         0.38412079, -0.2287056 ],
       [-0.27215874,  0.58515871, -0.81275526, ...,  0.31605803,
         0.38412079, -0.2287056 ],
       ...,
       [-0.27215874, -1.70893809, -0.81275526, ..., -3.16397598,
        -2.6033478 , -0.2287056 ],
       [-0.27215874, -1.70893809, -0.81275526, ..., -3.16397598,
        -2.6033478 , -0.2287056 ],
       [-0.27215874, -1.70893809, -0.81275526, ...,  0.31605803,
         0.38412079, -0.2287056 ]])

In [185]:
pcaOutput

,Person Code,Symbol,AsOnDate,General Ability Index,1_x,2_x,3_x,General Ability Index_scaled,1_y,2_y,3_y,FactorLoading1,FactorLoading2,FactorLoading3,FactorLoading4,FactorLoading5,FactorLoading6,FactorLoading7,FactorLoading8,FactorLoading9,FactorLoading10,FactorLoading11,FactorLoading12,FactorLoading13,FactorLoading14,FactorLoading15,FactorLoading16,FactorLoading17,FactorLoading18,FactorLoading19,FactorLoading20,Eigenvalue
0,P00027662,20MICRONS,2009-03-31,-1.981142,0.074610,-0.003697,-1.120158,-0.850692,0.065607,-0.003409,-1.055558,0.038186,0.133799,0.122044,0.231015,0.149687,0.13052,0.337257,0.208632,0.213774,0.341697,0.314974,0.269166,0.330243,0.05652,0.336136,0.044543,0.208647,0.217657,0.251816,0.002299,5.423606
1,P00027633,20MICRONS,2009-03-31,2.111865,-1.444338,0.542887,-0.980042,0.906824,-1.270047,0.500483,-0.923522,0.038186,0.133799,0.122044,0.231015,0.149687,0.13052,0.337257,0.208632,0.213774,0.341697,0.314974,0.269166,0.330243,0.05652,0.336136,0.044543,0.208647,0.217657,0.251816,0.002299,5.423606
2,P00027633,BANCOINDIA,2009-03-31,2.111865,-1.444338,0.542887,-0.980042,0.906824,-1.270047,0.500483,-0.923522,0.038186,0.133799,0.122044,0.231015,0.149687,0.13052,0.337257,0.208632,0.213774,0.341697,0.314974,0.269166,0.330243,0.05652,0.336136,0.044543,0.208647,0.217657,0.251816,0.002299,5.423606
3,P00027633,MUNJALAU,2009-03-31,2.111865,-1.444338,0.542887,-0.980042,0.906824,-1.270047,0.500483,-0.923522,0.038186,0.133799,0.122044,0.231015,0.149687,0.13052,0.337257,0.208632,0.213774,0.341697,0.314974,0.269166,0.330243,0.05652,0.336136,0.044543,0.208647,0.217657,0.251816,0.002299,5.423606
4,P00027664,20MICRONS,2009-03-31,-0.845518,1.084734,-0.647744,-0.247376,-0.363061,0.953837,-0.597149,-0.233109,0.038186,0.133799,0.122044,0.231015,0.149687,0.13052,0.337257,0.208632,0.213774,0.341697,0.314974,0.269166,0.330243,0.05652,0.336136,0.044543,0.208647,0.217657,0.251816,0.002299,5.423606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258252,P00081419,ZYLOG,2018-03-31,-4.462567,-1.350199,-0.172527,2.358387,-1.916203,-1.187268,-0.159051,2.222377,0.038186,0.133799,0.122044,0.231015,0.149687,0.13052,0.337257,0.208632,0.213774,0.341697,0.314974,0.269166,0.330243,0.05652,0.336136,0.044543,0.208647,0.217657,0.251816,0.002299,5.423606
258253,P00114429,ZYLOG,2018-03-31,-4.462567,-1.350199,-0.172527,2.358387,-1.916203,-1.187268,-0.159051,2.222377,0.038186,0.133799,0.122044,0.231015,0.149687,0.13052,0.337257,0.208632,0.213774,0.341697,0.314974,0.269166,0.330243,0.05652,0.336136,0.044543,0.208647,0.217657,0.251816,0.002299,5.423606
258254,P00080494,ZYLOG,2018-03-31,-4.462567,-1.350199,-0.172527,2.358387,-1.916203,-1.187268,-0.159051,2.222377,0.038186,0.133799,0.122044,0.231015,0.149687,0.13052,0.337257,0.208632,0.213774,0.341697,0.314974,0.269166,0.330243,0.05652,0.336136,0.044543,0.208647,0.217657,0.251816,0.002299,5.423606
258255,P00067111,ZYLOG,2018-03-31,-4.462567,-1.350199,-0.172527,2.358387,-1.916203,-1.187268,-0.159051,2.222377,0.038186,0.133799,0.122044,0.231015,0.149687,0.13052,0.337257,0.208632,0.213774,0.341697,0.314974,0.269166,0.330243,0.05652,0.336136,0.044543,0.208647,0.217657,0.251816,0.002299,5.423606


In [186]:
scaledData = pd.DataFrame(data = scaled_data, index = pcaData.index)\
.rename({0:"NumSkillsScaled", 1:"NumFirmsPastScaled", 2:"NumIndustryPastScaled", 3:"HasCeoMDChairXPScaled", 4:"HasConglomerateXPScaled"}, axis = 1)\
.reset_index(drop = False)
scaledData

,Person Code,Symbol,AsOnDate,NumSkillsScaled,NumFirmsPastScaled,NumIndustryPastScaled,HasCeoMDChairXPScaled,HasConglomerateXPScaled,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,P00027662,20MICRONS,2009-03-31,-0.272159,0.585159,-0.812755,-0.63427,-0.604166,-0.520576,-1.142448,-0.661226,-0.730998,-1.188985,-0.879696,-1.049473,1.120008,-0.205374,-1.016102,-0.13744,-0.839592,0.316058,0.384121,-0.228706
1,P00027633,20MICRONS,2009-03-31,-0.272159,0.585159,-0.812755,-0.63427,-0.604166,-0.520576,0.875314,-0.661226,1.367993,0.841054,1.136756,0.952859,1.120008,-0.205374,0.984153,-0.13744,1.191055,0.316058,0.384121,-0.228706
2,P00027633,BANCOINDIA,2009-03-31,-0.272159,0.585159,-0.812755,-0.63427,-0.604166,-0.520576,0.875314,-0.661226,1.367993,0.841054,1.136756,0.952859,1.120008,-0.205374,0.984153,-0.13744,1.191055,0.316058,0.384121,-0.228706
3,P00027633,MUNJALAU,2009-03-31,-0.272159,0.585159,-0.812755,-0.63427,-0.604166,-0.520576,0.875314,-0.661226,1.367993,0.841054,1.136756,0.952859,1.120008,-0.205374,0.984153,-0.13744,1.191055,0.316058,0.384121,-0.228706
4,P00027664,20MICRONS,2009-03-31,-0.272159,-1.708938,-0.812755,-0.63427,1.655175,-0.520576,0.875314,1.512341,-0.730998,-1.188985,1.136756,-1.049473,-0.892851,-0.205374,-1.016102,-0.13744,-0.839592,0.316058,0.384121,-0.228706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258252,P00081419,ZYLOG,2018-03-31,-0.272159,-1.708938,-0.812755,-0.63427,-0.604166,-0.520576,-1.142448,-0.661226,-0.730998,-1.188985,-0.879696,-1.049473,-0.892851,-0.205374,-1.016102,-0.13744,-0.839592,-3.163976,-2.603348,-0.228706
258253,P00114429,ZYLOG,2018-03-31,-0.272159,-1.708938,-0.812755,-0.63427,-0.604166,-0.520576,-1.142448,-0.661226,-0.730998,-1.188985,-0.879696,-1.049473,-0.892851,-0.205374,-1.016102,-0.13744,-0.839592,-3.163976,-2.603348,-0.228706
258254,P00080494,ZYLOG,2018-03-31,-0.272159,-1.708938,-0.812755,-0.63427,-0.604166,-0.520576,-1.142448,-0.661226,-0.730998,-1.188985,-0.879696,-1.049473,-0.892851,-0.205374,-1.016102,-0.13744,-0.839592,-3.163976,-2.603348,-0.228706
258255,P00067111,ZYLOG,2018-03-31,-0.272159,-1.708938,-0.812755,-0.63427,-0.604166,-0.520576,-1.142448,-0.661226,-0.730998,-1.188985,-0.879696,-1.049473,-0.892851,-0.205374,-1.016102,-0.13744,-0.839592,-3.163976,-2.603348,-0.228706


## Unscaled PCA

In [187]:
pcaData = mainDirect7[[
    'Person Code', 'Symbol', 'AsOnDate',
    'Academic','combined_manufacturing_supply_chain','Outside Board',
    'combined_sustainability', 'combined_entrepreneurial', 'combined_compensation',
    'combined_governance', 'combined_government_policy', 'combined_international',
    'combined_leadership', 'combined_legal',
    'combined_marketing', 'combined_risk_management', 'combined_scientific',
    'combined_strategic_planning', 'combined_conglomerate_experience',
    'combined_hr', 'combined_technology', 'combined_finance_accounting','Company Business' ]].copy().set_index(["Person Code", "Symbol", "AsOnDate"], drop = True)

In [188]:
scaled_data = pcaData
pca = PCA(n_components = 2)
pca.fit(scaled_data)
pca_data = pca.transform(scaled_data)

pca_data_scaled = preprocessing.scale(pca_data)


eigenvalues = pca.explained_variance_
eigenvectors = pca.components_

In [189]:
pcaOutputNonScaled = pd.DataFrame(data = pca_data, index = pcaData.index)\
.rename({0:"General Ability Index"}, axis = 1).reset_index(drop = False)

pcaOutputScaled = pd.DataFrame(data = pca_data_scaled, index = pcaData.index)\
.rename({0:"General Ability Index_scaled"}, axis = 1).reset_index(drop = False)

pcaOutput = pcaOutputNonScaled.merge(pcaOutputScaled, on = ["Person Code", "Symbol", "AsOnDate"], how = "inner")
pcaOutput

,Person Code,Symbol,AsOnDate,General Ability Index,1_x,General Ability Index_scaled,1_y
0,P00027662,20MICRONS,2009-03-31,-1.041318,-0.200416,-0.946700,-0.411542
1,P00027633,20MICRONS,2009-03-31,1.109191,-0.941648,1.008406,-1.933621
2,P00027633,BANCOINDIA,2009-03-31,1.109191,-0.941648,1.008406,-1.933621
3,P00027633,MUNJALAU,2009-03-31,1.109191,-0.941648,1.008406,-1.933621
4,P00027664,20MICRONS,2009-03-31,-0.487280,0.373439,-0.443005,0.766835
...,...,...,...,...,...,...,...
258252,P00081419,ZYLOG,2018-03-31,-1.787118,-0.122477,-1.624735,-0.251499
258253,P00114429,ZYLOG,2018-03-31,-1.787118,-0.122477,-1.624735,-0.251499
258254,P00080494,ZYLOG,2018-03-31,-1.787118,-0.122477,-1.624735,-0.251499
258255,P00067111,ZYLOG,2018-03-31,-1.787118,-0.122477,-1.624735,-0.251499


In [190]:
explained_variance = pca.explained_variance_ratio_
explained_variance

array([0.34347847, 0.06732637])

In [191]:
eigenvectors

array([[ 0.01579989,  0.12035961,  0.12798216,  0.21793847,  0.13024752,
         0.10379515,  0.35702792,  0.20311827,  0.21480433,  0.36312512,
         0.34088726,  0.28978788,  0.35688422,  0.01857987,  0.36585608,
         0.01048559,  0.21902011,  0.11150747,  0.15704952,  0.00107582],
       [ 0.01508148, -0.1240447 ,  0.47171205,  0.31648269,  0.46259763,
         0.52087957,  0.15437062, -0.08182337, -0.11591918, -0.11559548,
        -0.12563301, -0.19736902, -0.04029778,  0.01910459, -0.15905962,
        -0.00150476, -0.1820272 ,  0.04496777,  0.04143576,  0.0131401 ]])

In [192]:
pcaOutput["General Ability Index"].var()

1.209882710068733

In [193]:
pcaOutput["FactorLoading1"] = eigenvectors[0, 0]
pcaOutput["FactorLoading2"] = eigenvectors[0, 1]
pcaOutput["FactorLoading3"] = eigenvectors[0, 2]
pcaOutput["FactorLoading4"] = eigenvectors[0, 3]
pcaOutput["FactorLoading5"] = eigenvectors[0, 4]

pcaOutput["FactorLoading6"] = eigenvectors[0, 5]
pcaOutput["FactorLoading7"] = eigenvectors[0, 6]
pcaOutput["FactorLoading8"] = eigenvectors[0, 7]
pcaOutput["FactorLoading9"] = eigenvectors[0, 8]
pcaOutput["FactorLoading10"] = eigenvectors[0, 9]

pcaOutput["FactorLoading11"] = eigenvectors[0, 10]
pcaOutput["FactorLoading12"] = eigenvectors[0, 11]
pcaOutput["FactorLoading13"] = eigenvectors[0, 12]
pcaOutput["FactorLoading14"] = eigenvectors[0, 13]
pcaOutput["FactorLoading15"] = eigenvectors[0, 14]

pcaOutput["FactorLoading16"] = eigenvectors[0, 15]
pcaOutput["FactorLoading17"] = eigenvectors[0, 16]
pcaOutput["FactorLoading18"] = eigenvectors[0, 17]
pcaOutput["FactorLoading19"] = eigenvectors[0, 18]
pcaOutput["FactorLoading20"] = eigenvectors[0, 19]

pcaOutput["Eigenvalue"] = eigenvalues[0]

In [194]:
scaled_data

Academic  \
Person Code Symbol     AsOnDate               
P00027662   20MICRONS  2009-03-31         0   
P00027633   20MICRONS  2009-03-31         0   
            BANCOINDIA 2009-03-31         0   
            MUNJALAU   2009-03-31         0   
P00027664   20MICRONS  2009-03-31         0   
...                                     ...   
P00081419   ZYLOG      2018-03-31         0   
P00114429   ZYLOG      2018-03-31         0   
P00080494   ZYLOG      2018-03-31         0   
P00067111   ZYLOG      2018-03-31         0   
P00075399   ZYLOG      2018-03-31         0   

                                   combined_manufacturing_supply_chain  \
Person Code Symbol     AsOnDate                                          
P00027662   20MICRONS  2009-03-31                                    1   
P00027633   20MICRONS  2009-03-31                                    1   
            BANCOINDIA 2009-03-31                                    1   
            MUNJALAU   2009-03-31                                    1   
P00027664   20MICRONS  2009-03-31                                    0   
...                                                                ...   
P00081419   ZYLOG      2018-03-31                                    0   
P00114429   ZYLOG      2018-03-31                                    0   
P00080494   ZYLOG      2018-03-31                                    0   
P00067111   ZYLOG      2018-03-31                                    0   
P00075399   ZYLOG      2018-03-31                                    0   

                                   Outside Board  combined_sustainability  \
Person Code Symbol     AsOnDate                                             
P00027662   20MICRONS  2009-03-31              0                        0   
P00027633   20MICRONS  2009-03-31              0                        0   
            BANCOINDIA 2009-03-31              0                        0   
            MUNJALAU   2009-03-31              0                        0   
P00027664   20MICRONS  2009-03-31              0                        0   
...                                          ...                      ...   
P00081419   ZYLOG      2018-03-31              0                        0   
P00114429   ZYLOG      2018-03-31              0                        0   
P00080494   ZYLOG      2018-03-31              0                        0   
P00067111   ZYLOG      2018-03-31              0                        0   
P00075399   ZYLOG      2018-03-31              0                        0   

                                   combined_entrepreneurial  \
Person Code Symbol     AsOnDate                               
P00027662   20MICRONS  2009-03-31                         0   
P00027633   20MICRONS  2009-03-31                         0   
            BANCOINDIA 2009-03-31                         0   
            MUNJALAU   2009-03-31                         0   
P00027664   20MICRONS  2009-03-31                         1   
...                                                     ...   
P00081419   ZYLOG      2018-03-31                         0   
P00114429   ZYLOG      2018-03-31                         0   
P00080494   ZYLOG      2018-03-31                         0   
P00067111   ZYLOG      2018-03-31                         0   
P00075399   ZYLOG      2018-03-31                         0   

                                   combined_compensation  combined_governance  \
Person Code Symbol     AsOnDate                                                 
P00027662   20MICRONS  2009-03-31                      0                    0   
P00027633   20MICRONS  2009-03-31                      0                    1   
            BANCOINDIA 2009-03-31                      0                    1   
            MUNJALAU   2009-03-31                      0                    1   
P00027664   20MICRONS  2009-03-31                      0                    1   
...                                                  ...               

In [195]:
pcaOutput

,Person Code,Symbol,AsOnDate,General Ability Index,1_x,General Ability Index_scaled,1_y,FactorLoading1,FactorLoading2,FactorLoading3,FactorLoading4,FactorLoading5,FactorLoading6,FactorLoading7,FactorLoading8,FactorLoading9,FactorLoading10,FactorLoading11,FactorLoading12,FactorLoading13,FactorLoading14,FactorLoading15,FactorLoading16,FactorLoading17,FactorLoading18,FactorLoading19,FactorLoading20,Eigenvalue
0,P00027662,20MICRONS,2009-03-31,-1.041318,-0.200416,-0.946700,-0.411542,0.0158,0.12036,0.127982,0.217938,0.130248,0.103795,0.357028,0.203118,0.214804,0.363125,0.340887,0.289788,0.356884,0.01858,0.365856,0.010486,0.21902,0.111507,0.15705,0.001076,1.209883
1,P00027633,20MICRONS,2009-03-31,1.109191,-0.941648,1.008406,-1.933621,0.0158,0.12036,0.127982,0.217938,0.130248,0.103795,0.357028,0.203118,0.214804,0.363125,0.340887,0.289788,0.356884,0.01858,0.365856,0.010486,0.21902,0.111507,0.15705,0.001076,1.209883
2,P00027633,BANCOINDIA,2009-03-31,1.109191,-0.941648,1.008406,-1.933621,0.0158,0.12036,0.127982,0.217938,0.130248,0.103795,0.357028,0.203118,0.214804,0.363125,0.340887,0.289788,0.356884,0.01858,0.365856,0.010486,0.21902,0.111507,0.15705,0.001076,1.209883
3,P00027633,MUNJALAU,2009-03-31,1.109191,-0.941648,1.008406,-1.933621,0.0158,0.12036,0.127982,0.217938,0.130248,0.103795,0.357028,0.203118,0.214804,0.363125,0.340887,0.289788,0.356884,0.01858,0.365856,0.010486,0.21902,0.111507,0.15705,0.001076,1.209883
4,P00027664,20MICRONS,2009-03-31,-0.487280,0.373439,-0.443005,0.766835,0.0158,0.12036,0.127982,0.217938,0.130248,0.103795,0.357028,0.203118,0.214804,0.363125,0.340887,0.289788,0.356884,0.01858,0.365856,0.010486,0.21902,0.111507,0.15705,0.001076,1.209883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258252,P00081419,ZYLOG,2018-03-31,-1.787118,-0.122477,-1.624735,-0.251499,0.0158,0.12036,0.127982,0.217938,0.130248,0.103795,0.357028,0.203118,0.214804,0.363125,0.340887,0.289788,0.356884,0.01858,0.365856,0.010486,0.21902,0.111507,0.15705,0.001076,1.209883
258253,P00114429,ZYLOG,2018-03-31,-1.787118,-0.122477,-1.624735,-0.251499,0.0158,0.12036,0.127982,0.217938,0.130248,0.103795,0.357028,0.203118,0.214804,0.363125,0.340887,0.289788,0.356884,0.01858,0.365856,0.010486,0.21902,0.111507,0.15705,0.001076,1.209883
258254,P00080494,ZYLOG,2018-03-31,-1.787118,-0.122477,-1.624735,-0.251499,0.0158,0.12036,0.127982,0.217938,0.130248,0.103795,0.357028,0.203118,0.214804,0.363125,0.340887,0.289788,0.356884,0.01858,0.365856,0.010486,0.21902,0.111507,0.15705,0.001076,1.209883
258255,P00067111,ZYLOG,2018-03-31,-1.787118,-0.122477,-1.624735,-0.251499,0.0158,0.12036,0.127982,0.217938,0.130248,0.103795,0.357028,0.203118,0.214804,0.363125,0.340887,0.289788,0.356884,0.01858,0.365856,0.010486,0.21902,0.111507,0.15705,0.001076,1.209883


In [196]:
scaledData = pd.DataFrame(data = scaled_data, index = pcaData.index)\
.rename({0:"NumSkillsScaled", 1:"NumFirmsPastScaled", 2:"NumIndustryPastScaled", 3:"HasCeoMDChairXPScaled", 4:"HasConglomerateXPScaled"}, axis = 1)\
.reset_index(drop = False)
scaledData

,Person Code,Symbol,AsOnDate,Academic,combined_manufacturing_supply_chain,Outside Board,combined_sustainability,combined_entrepreneurial,combined_compensation,combined_governance,combined_government_policy,combined_international,combined_leadership,combined_legal,combined_marketing,combined_risk_management,combined_scientific,combined_strategic_planning,combined_conglomerate_experience,combined_hr,combined_technology,combined_finance_accounting,Company Business
0,P00027662,20MICRONS,2009-03-31,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0
1,P00027633,20MICRONS,2009-03-31,0,1,0,0,0,0,1,0,1,1,1,1,1,0,1,0,1,1,1,0
2,P00027633,BANCOINDIA,2009-03-31,0,1,0,0,0,0,1,0,1,1,1,1,1,0,1,0,1,1,1,0
3,P00027633,MUNJALAU,2009-03-31,0,1,0,0,0,0,1,0,1,1,1,1,1,0,1,0,1,1,1,0
4,P00027664,20MICRONS,2009-03-31,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258252,P00081419,ZYLOG,2018-03-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
258253,P00114429,ZYLOG,2018-03-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
258254,P00080494,ZYLOG,2018-03-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
258255,P00067111,ZYLOG,2018-03-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### PCA for only 2013 and onwards

In [197]:
pcaData2013 = mainDirect7.loc[ mainDirect7["AsOnDate"] >= "2013-01-01",
["Person Code", "Symbol", "AsOnDate", "NumSkills", "NumFirmsPast", "NumIndustryPast", "HasCeoMDChairXP", "HasConglomerateXP"]].copy().set_index(["Person Code", "Symbol", "AsOnDate"], drop = True)

In [198]:
scaled_data2013 = preprocessing.scale(pcaData2013)
pca2013 = PCA(n_components = 1)
pca2013.fit(scaled_data2013)
pca_data2013 = pca2013.transform(scaled_data2013)

pca_data_scaled2013 = preprocessing.scale(pca_data2013)

eigenvalues2013 = pca2013.explained_variance_
eigenvectors2013 = pca2013.components_

In [199]:
pcaOutputNonScaled2013 = pd.DataFrame(data = pca_data2013, index = pcaData2013.index)\
.rename({0:"General Ability Index_2013"}, axis = 1).reset_index(drop = False)

pcaOutputScaled2013 = pd.DataFrame(data = pca_data_scaled2013, index = pcaData2013.index)\
.rename({0:"General Ability Index_scaled_2013"}, axis = 1).reset_index(drop = False)

pcaOutput2013 = pcaOutputNonScaled2013.merge(pcaOutputScaled2013, on = ["Person Code", "Symbol", "AsOnDate"], how = "inner")
pcaOutput2013

,Person Code,Symbol,AsOnDate,General Ability Index_2013,General Ability Index_scaled_2013
0,P00027633,20MICRONS,2013-03-31,1.284537,0.819956
1,P00027633,BANCOINDIA,2013-03-31,1.284537,0.819956
2,P00027633,MUNJALAU,2013-03-31,1.284537,0.819956
3,P00042601,20MICRONS,2013-03-31,-0.873932,-0.557855
4,P00029006,20MICRONS,2013-03-31,-0.068427,-0.043679
...,...,...,...,...,...
185139,P00081419,ZYLOG,2018-03-31,-1.593633,-1.017260
185140,P00114429,ZYLOG,2018-03-31,-1.593633,-1.017260
185141,P00080494,ZYLOG,2018-03-31,-1.593633,-1.017260
185142,P00067111,ZYLOG,2018-03-31,-1.593633,-1.017260


In [200]:
pcaOutput2013["FactorLoading1_2013"] = eigenvectors[0, 0]
pcaOutput2013["FactorLoading2_2013"] = eigenvectors[0, 1]
pcaOutput2013["FactorLoading3_2013"] = eigenvectors[0, 2]
pcaOutput2013["FactorLoading4_2013"] = eigenvectors[0, 3]
pcaOutput2013["FactorLoading5_2013"] = eigenvectors[0, 4]

pcaOutput2013["Eigenvalue2013"] = eigenvalues[0]

In [201]:
scaled_data2013

array([[ 1.37371223,  0.41943258,  0.51684823, -0.31921273,  0.57454786],
       [ 1.37371223,  0.41943258,  0.51684823, -0.31921273,  0.57454786],
       [ 1.37371223,  0.41943258,  0.51684823, -0.31921273,  0.57454786],
       ...,
       [-1.19572186, -0.42847292, -0.41872483, -0.31921273, -1.74049903],
       [-1.19572186, -0.42847292, -0.41872483, -0.31921273, -1.74049903],
       [-1.19572186, -0.42847292, -0.41872483, -0.31921273, -1.74049903]])

In [202]:
pcaOutput2013

,Person Code,Symbol,AsOnDate,General Ability Index_2013,General Ability Index_scaled_2013,FactorLoading1_2013,FactorLoading2_2013,FactorLoading3_2013,FactorLoading4_2013,FactorLoading5_2013,Eigenvalue2013
0,P00027633,20MICRONS,2013-03-31,1.284537,0.819956,0.0158,0.12036,0.127982,0.217938,0.130248,1.209883
1,P00027633,BANCOINDIA,2013-03-31,1.284537,0.819956,0.0158,0.12036,0.127982,0.217938,0.130248,1.209883
2,P00027633,MUNJALAU,2013-03-31,1.284537,0.819956,0.0158,0.12036,0.127982,0.217938,0.130248,1.209883
3,P00042601,20MICRONS,2013-03-31,-0.873932,-0.557855,0.0158,0.12036,0.127982,0.217938,0.130248,1.209883
4,P00029006,20MICRONS,2013-03-31,-0.068427,-0.043679,0.0158,0.12036,0.127982,0.217938,0.130248,1.209883
...,...,...,...,...,...,...,...,...,...,...,...
185139,P00081419,ZYLOG,2018-03-31,-1.593633,-1.017260,0.0158,0.12036,0.127982,0.217938,0.130248,1.209883
185140,P00114429,ZYLOG,2018-03-31,-1.593633,-1.017260,0.0158,0.12036,0.127982,0.217938,0.130248,1.209883
185141,P00080494,ZYLOG,2018-03-31,-1.593633,-1.017260,0.0158,0.12036,0.127982,0.217938,0.130248,1.209883
185142,P00067111,ZYLOG,2018-03-31,-1.593633,-1.017260,0.0158,0.12036,0.127982,0.217938,0.130248,1.209883


In [203]:
scaledData2013 = pd.DataFrame(data = scaled_data2013, index = pcaData2013.index)\
.rename({0:"NumSkillsScaled_2013", 1:"NumFirmsPastScaled_2013", 2:"NumIndustryPastScaled_2013", 3:"HasCeoMDChairXPScaled_2013", 4:"HasConglomerateXPScaled_2013"}, axis = 1)\
.reset_index(drop = False)
scaledData2013

,Person Code,Symbol,AsOnDate,NumSkillsScaled_2013,NumFirmsPastScaled_2013,NumIndustryPastScaled_2013,HasCeoMDChairXPScaled_2013,HasConglomerateXPScaled_2013
0,P00027633,20MICRONS,2013-03-31,1.373712,0.419433,0.516848,-0.319213,0.574548
1,P00027633,BANCOINDIA,2013-03-31,1.373712,0.419433,0.516848,-0.319213,0.574548
2,P00027633,MUNJALAU,2013-03-31,1.373712,0.419433,0.516848,-0.319213,0.574548
3,P00042601,20MICRONS,2013-03-31,-1.195722,-0.428473,-0.418725,-0.319213,0.574548
4,P00029006,20MICRONS,2013-03-31,0.688530,-0.428473,-0.418725,-0.319213,0.574548
...,...,...,...,...,...,...,...,...
185139,P00081419,ZYLOG,2018-03-31,-1.195722,-0.428473,-0.418725,-0.319213,-1.740499
185140,P00114429,ZYLOG,2018-03-31,-1.195722,-0.428473,-0.418725,-0.319213,-1.740499
185141,P00080494,ZYLOG,2018-03-31,-1.195722,-0.428473,-0.418725,-0.319213,-1.740499
185142,P00067111,ZYLOG,2018-03-31,-1.195722,-0.428473,-0.418725,-0.319213,-1.740499


## 5 Merge with Main 6

In [204]:
mainDirect8 = mainDirect7.merge(scaledData, on = ["Person Code", "Symbol", "AsOnDate"], how = "inner")
mainDirect9 = mainDirect8.merge(pcaOutput, on = ["Person Code", "Symbol", "AsOnDate"], how = "inner")

mainDirect10 = mainDirect9.merge(scaledData2013, on = ["Person Code", "Symbol", "AsOnDate"], how = "left")
mainDirect11 = mainDirect10.merge(pcaOutput2013, on = ["Person Code", "Symbol", "AsOnDate"], how = "left")

In [205]:
mainDirect11

,Symbol,Company,AsOnDate,AsOnYear,ISIN,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Member of Civil Services,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Occupation,Cessation Reason,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Brief Profile,Tenure Valid till,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,AppointDummy,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal,CloBalTotalXP,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMDPosition,IsChairmanPosition,IsCeoMDOccupation,IsChairmanOccupation,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManager,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsinCompIndep,TenureInYearsinCompTotal,IsFirstTerm,IsFirstTermIndep,IsZeroYear,IsZeroYearIndep,IsOneYear,IsOneYearIndep,IsTwoYear,IsTwoYearIndep,IsThreeYear,IsThreeYearIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,IsBusy,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,NSE symbol,NIC code,Entity type,Ownership group,Prowess company code,Company Name,Skills,EducationAll,IsMBA,IsPhD,AllPastDirectNIC,CurrDirectANIC,CurrDirectABNIC,AllNIC,AllNIC_Industry,HasFinanceXP,HasTechXP,HasRelatedIndustryXP,IsExecCurrent,NumExecAll,IsOutsideExecXP,HasExecXP,PublicExecXPDummy,PrivateExecXPDummy,HasPublicExecXP,HasPrivateExecXP,SkillsInPositiononBoard,SkillsInOccupation,AllSkills,HasTechSkill,HasFinanceSkill,FirmsPast,NumSkills,NumFirmsPast,NumIndustryPast,HasCeoMDChairXP,HasConglomerateXP,skilllist_sustainability,skilllist_entrepreneurial,skilllist_compensation,skilllist_conglomerate_experience,skilllist_hr,skilllist_technology,skilllist_finance_accounting,skilllist_governance,skilllist_government_policy,skilllist_international,skilllist_leadership,skilllist_legal,skilllist_marketing,skilllist_risk_management,skilllist_scientific,skilllist_strategic_planning,skilllist_manufacturing_supply_chain,Academic_x,Manufacturing,Outside Board_x,Company Business_x,profile_sustainability,profile_entrepreneurial,profile_compensation,profile_conglomerate_experience,profile_hr,profile_technology,profile_finance_accounting,profile_governance,profile_government_policy,profile_international,profile_leadership,profile_legal,profile_marketing,profile_risk_management,profile_scientific,profile_strategic_planning,profile_manufacturing_supply_chain,Committee Name,committee_sustainability,committee_entrepreneurial,committee_compensation,committee_conglomerate_experience,committee_hr,committee_technology,committee_finance_accounting,committee_governance,committee_government_policy,committee_international,committee_leadership,committee_legal,committee_mark

# EXPORTING UPDATES

## Update 1: PCA output

In [206]:
# update1col = ["Symbol", "Person Code", "AsOnDate",
#               "NumSkillsScaled", "NumFirmsPastScaled", "NumIndustryPastScaled", "HasCeoMDChairXPScaled", "HasConglomerateXPScaled",
#               "General Ability Index",
#               "FactorLoading1", "FactorLoading2", "FactorLoading3", "FactorLoading4", "FactorLoading5", "Eigenvalue"]

# update1 = mainDirect8.loc[:, update1col].copy()
# update1

In [207]:
# update1.to_csv("Main_Director_FACTOR SCORES_UPDATE1_v270325.csv")